In [3]:
#Import dependencies
import streamlit as st
import datetime
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import pyfolio as pf

# Set display options for Pandas
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

/Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [4]:
# Function to fetch asset data using yfinance
def fetch_asset_data(api_pull, start_date, end_date):
    data_frames = [] # Initialize an empty list to store data frames

    # Iterate over asset classes and their corresponding tickers
    for asset_class, tickers in api_pull.items():
        for ticker in tickers:
            # Download historical data for each ticker within the specified date range
            historical_data = yf.download(ticker, start=start_date, end=end_date)
            historical_data['Asset Class'] = asset_class # Add a column indicating the asset class
            historical_data['Ticker'] = ticker # Add a column indicating the ticker
            data_frames.append(historical_data) # Append the data frame to the list

    # Concatenate all data frames in the list into a single data frame
    combined_data = pd.concat(data_frames)
    return combined_data

In [5]:
# Streamlit app
st.title("Portfolio Optimization")

AttributeError: module 'streamlit' has no attribute 'title'

In [6]:
# Define tickers for different asset classes

# VAR1.1 1 with 10 individial tickers and commmodities for Jupyter lab 
# crypto_tickers = ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
# stocks_tickers = ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META', 'XOM', 'LAC']
# commodities_tickers = ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']
# bonds_tickers = ['TLT', 'IEF', 'SHY', 'AGG', 'BND', 'LQD', 'HYG', 'MUB', 'TIP', 'EMB']

# VAR 1.2 with 20  individial tickers for Streamlit with individual stocks and bonds
# Prompt user to choose assets in asset classes
# crypto_selected = st.multiselect("Choose cryptocurrencies:", options=['AAVE-USD', 'ALGO-USD', 'BAT-USD', 'BCH-USD', 'BTC-USD', 'DAI-USD', 'ETH-USD', 'GRT-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NEAR-USD', 'PAXG-USD', 'SHIB-USD', 'SOL-USD', 'TRX-USD', 'UNI-USD', 'USDT-USD ', 'WBTC-USD'])
# stocks_selected = st.multiselect("Choose stocks:", options=['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'JPM ', 'V', 'JNJ', 'UNH', 'PG', 'KO', 'XOM', 'CVX', 'BA', 'CAT', 'NVDA', 'NVDA','DIS', 'PFE'])
# commodities_selected = st.multiselect("Choose commodities:", options=['GLD', 'SLV', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F'])not supported on Alpaca
# bonds_selected = st.multiselect("Choose bonds:", options=['TLT', 'IEF', 'SHY', 'AGG', 'BND', 'LQD', 'HYG', 'MUB', 'TIP', 'EMB'])not supported on Alpaca

# VAR1.1 3 with with 20  individial tickers for Jupyter lab 
# crypto_tickers = ['AAVE-USD', 'ALGO-USD', 'BAT-USD', 'BCH-USD', 'BTC-USD', 'DAI-USD', 'ETH-USD', 'GRT-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NEAR-USD', 'PAXG-USD', 'SHIB-USD', 'SOL-USD', 'TRX-USD', 'UNI-USD', 'USDT-USD ', 'WBTC-USD']
# stocks_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'JPM ', 'V', 'JNJ', 'UNH', 'PG', 'KO', 'XOM', 'CVX', 'BA', 'CAT', 'NVDA', 'NVDA','DIS', 'PFE']
# commodities_tickers = ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']not supported on Alpaca
# bonds_tickers = ['TLT', 'IEF', 'SHY', 'AGG', 'BND', 'LQD', 'HYG', 'MUB', 'TIP', 'EMB']not supported on Alpaca


# VAR 2.1 with 20 tickers and ETF's for Streamlit with crypto and ETF's dsupported USD pairs by Alpaca
# (USDT_USD is neded just for swaping or preserving purchasing power)
# Prompt user to choose assets in asset classes
# crypto_selected = st.multiselect("Choose cryptocurrencies:", options=['AAVE-USD', 'ALGO-USD', 'BAT-USD', 'BCH-USD', 'BTC-USD', 'DAI-USD', 'ETH-USD', 'GRT-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NEAR-USD', 'PAXG-USD', 'SHIB-USD', 'SOL-USD', 'TRX-USD', 'UNI-USD', 'USDT-USD ', 'WBTC-USD'])
# stocks_selected = st.multiselect("Choose stocks:", options=['SPY', 'QQQ', 'IWM', 'VTI', 'VOO', 'VO', 'VB', 'VEA', 'VWO', 'XLF ', 'XLV', 'XLE', 'XLY', 'XLC', 'XLK', 'XLI', 'XLP', 'XLB', 'XLU', 'XLRE'])                                
# commodities_selected = st.multiselect("Choose commodities:", options=['GLD', 'SLV', 'USO', 'UNG', 'DBA', 'DBC', 'GSG', 'IAU ', 'PPLT', 'SIVR', 'MOO', 'NIB', 'JO', 'JJG', 'WEAT', 'UGA', 'DBE', 'HAP', 'REMX', 'OIL'])
# bonds_selected = st.multiselect("Choose bonds:", options=['AGG', 'BND', 'TLT', 'IEF', 'SHY', 'LQD', 'HYG', 'JNK', 'MUB', 'TIP', 'BNDX', 'EMB', 'VWOB', 'PFF', 'BKLN', 'FLOT', 'GSY', 'SCHO', 'SCHR', 'SCHZ'])                                    
                                      
# VAR 2.2 with 20 tickers and commmodities for Jupyter lab with crypto and ETF's dsupported USD pairs by Alpaca
# (USDT_USD is neded just for swaping or preserving purchasing power) 
# WITH REMOVED NAN TICKERS !!!
# Prompt user to choose assets in asset classes
crypto_tickers = ['AAVE-USD', 'ALGO-USD', 'BAT-USD', 'BCH-USD', 'BTC-USD', 'DAI-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NEAR-USD', 'PAXG-USD', 'SOL-USD', 'TRX-USD', 'USDT-USD ', 'WBTC-USD'] 
stocks_tickers = ['SPY', 'QQQ', 'IWM', 'VTI', 'VOO', 'VO', 'VB', 'VEA', 'VWO', 'XLF ', 'XLV', 'XLE', 'XLY', 'XLC', 'XLK', 'XLI', 'XLP', 'XLB', 'XLU', 'XLRE']
commodities_tickers = ['GLD', 'SLV', 'USO', 'UNG', 'DBA', 'DBC', 'GSG', 'IAU ', 'PPLT', 'SIVR', 'MOO', 'NIB', 'JO', 'JJG', 'WEAT', 'UGA', 'DBE', 'REMX', 'OIL']
bonds_tickers = ['AGG', 'BND', 'TLT', 'IEF', 'SHY', 'LQD', 'HYG', 'JNK', 'MUB', 'TIP', 'BNDX', 'EMB', 'VWOB', 'PFF', 'BKLN', 'FLOT', 'GSY', 'SCHO', 'SCHR', 'SCHZ']



Create a DataFrame with the tickers and their corresponding first trading dates.

In [7]:
# # Define lists of tickers for different asset classes
# crypto_tickers = ['AAVE-USD', 'ALGO-USD', 'BAT-USD', 'BCH-USD', 'BTC-USD', 'DAI-USD', 'ETH-USD', 'GRT-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NEAR-USD', 'PAXG-USD', 'SHIB-USD', 'SOL-USD', 'TRX-USD', 'UNI-USD', 'USDT-USD', 'WBTC-USD']
# stocks_tickers = ['SPY', 'QQQ', 'IWM', 'VTI', 'VOO', 'VO', 'VB', 'VEA', 'VWO', 'XLF', 'XLV', 'XLE', 'XLY', 'XLC', 'XLK', 'XLI', 'XLP', 'XLB', 'XLU', 'XLRE']
# commodities_tickers = ['GLD', 'SLV', 'USO', 'UNG', 'DBA', 'DBC', 'GSG', 'IAU', 'PPLT', 'SIVR', 'MOO', 'NIB', 'JO', 'JJG', 'WEAT', 'UGA', 'DBE', 'HAP', 'REMX', 'OIL']
# bonds_tickers = ['AGG', 'BND', 'TLT', 'IEF', 'SHY', 'LQD', 'HYG', 'JNK', 'MUB', 'TIP', 'BNDX', 'EMB', 'VWOB', 'PFF', 'BKLN', 'FLOT', 'GSY', 'SCHO', 'SCHR', 'SCHZ']

# Combine all tickers into a single list
all_tickers = crypto_tickers + stocks_tickers + commodities_tickers + bonds_tickers

first_trading_dates = []

# Iterate through each ticker
for ticker in all_tickers:
    # Download the ticker's historical data
    data = yf.download(ticker, start='2000-01-01', end='2023-04-06', progress=False)
    # Extract the first trading date
    first_date = data.index[0].strftime('%Y-%m-%d')
    # Append the ticker and its first trading date to the list
    first_trading_dates.append((ticker, first_date))

# Create a Pandas DataFrame with the tickers and their corresponding first trading dates
df = pd.DataFrame(first_trading_dates, columns=['Ticker', 'First Trading Date'])









In [8]:
#Display whole df and sort the firts tradig date 
pd.options.display.max_rows = None
df = df.sort_values(by='First Trading Date', ascending=True)
df

,Ticker,First Trading Date
28,XLE,2000-01-03
26,XLF,2000-01-03
27,XLV,2000-01-03
17,SPY,2000-01-03
29,XLY,2000-01-03
31,XLK,2000-01-03
18,QQQ,2000-01-03
33,XLP,2000-01-03
34,XLB,2000-01-03
35,XLU,2000-01-03


In [9]:
# VAR 1 for Streamlit 
# Define the tickers for each asset class.
# api_pull = {'crypto': crypto_selected,
#             'stocks': stocks_selected,
#             'commodities': commodities_selected,
#             'bonds': bonds_selected}

# VAR 2 for Jupyter lab 
# Define the tickers for each asset class.
api_pull = {'crypto': crypto_tickers,
            'stocks': stocks_tickers,
            'commodities': commodities_tickers,
            'bonds': bonds_tickers}

In [10]:
# Prompt user to choose time period
st.write("Choose the analysis period:\n"
          "Note that you can only choose a period starting from Jan 1st, 2020!")

AttributeError: module 'streamlit' has no attribute 'write'

In [ ]:
# FOR STREAMLIT
# # Get the current date
# today = datetime.datetime.now().date()

# # Set the earliest allowed start date to January 1st, 2020
# earliest_start_date = datetime.date(2020, 1, 1)

# # Get the user selected start and end dates
# selected_start_date = st.date_input("Select the start date", earliest_start_date)

# # If the selected start date is earlier than the earliest allowed start date, set it to the earliest start date
# if selected_start_date < earliest_start_date:
#     selected_start_date = earliest_start_date

# selected_end_date = st.date_input("Select the end date", today)

# # If the selected end date is later than the current date, set it to the current date
# if selected_end_date > today:
#     selected_end_date = today

# # If the selected end date is earlier than the selected start date, set the end date to the start date
# if selected_end_date < selected_start_date:
#     selected_end_date = selected_start_date


In [ ]:
# FOR STREAMLIT
# #Call fetch_asset_data function to pull data from yfinance API
# data = fetch_asset_data(api_pull, selected_start_date, selected_end_date)

# #Display the fetched data
# st.write("Fetched Data:")
# st.dataframe(data)

# Define the time range for the data
selected_start_date = '2020-10-14'
selected_end_date = '2023-03-22'

data = fetch_asset_data(api_pull, selected_start_date, selected_end_date)

In [ ]:
data.head()


In [12]:
#Reset the index of a DataFrame and set a new index with multiple columns
data = data.reset_index().set_index(['Asset Class', 'Ticker','Date'])

In [13]:
display(data.head(30))
data.tail(10)

Open       High        Low      Close  Adj Close     Volume
Asset Class Ticker   Date                                                                        
crypto      AAVE-USD 2020-10-14  51.372898  57.853947  49.629528  51.316517  51.316517    3132405
                     2020-10-15  51.387611  52.080685  44.018326  44.019184  44.019184    5883146
                     2020-10-16  42.890606  43.906731  39.142956  39.495846  39.495846   52855205
                     2020-10-17  39.486191  43.077374  38.633614  41.366993  41.366993   43339687
                     2020-10-18  41.389599  41.749489  39.531586  40.746971  40.746971   40441163
                     2020-10-19  40.836758  40.995186  35.951363  36.012318  36.012318   49289897
                     2020-10-20  36.043785  36.319069  32.101101  32.180016  32.180016   60632326
                     2020-10-21  32.253490  37.078201  32.048149  35.160358  35.160358   43831235
                     2020-10-22  35.097488  39.856922  33.589462  38.031651  38.031651   54717366
                     2020-10-23  37.730595  42.546658  36.924671  41.686638  41.686638   76081860
                     2020-10-24  41.686638  42.439705  38.728821  39.455021  39.455021   57658170
                     2020-10-25  39.455021  40.928509  37.601200  37.904762  37.904762   43970452
                     2020-10-26  37.904762  39.163864  35.071968  36.033920  36.033920   50307223
                     2020-10-27  36.033913  38.869183  34.469635  34.871532  34.871532   58581308
                     2020-10-28  34.871536  35.564022  32.000057  32.394081  32.394081   60533867
                     2020-10-29  32.394081  33.155483  28.240772  29.098333  29.098333   66304786
                     2020-10-30  29.098341  31.980774  27.792461  31.217823  31.217823   77280724
                     2020-10-31  31.217823  32.900162  29.610258  29.707518  29.707518   58631072
                     2020-11-01  29.707518  31.067060  28.494482  30.657276  30.657276   49568339
                     2020-11-02  30.657282  33.923584  29.344650  29.731524  29.731524   87180146
                     2020-11-03  29.731525  30.062918  27.537111  28.042803  28.042803   56775012
                     2020-11-04  28.042971  28.433069  26.089518  27.715084  27.715084   50464691
                     2020-11-05  27.715080  32.303040  25.966154  30.955627  30.955627   85212412
                     2020-11-06  30.955118  40.769073  30.855127  38.457439  38.457439  147304661
                     2020-11-07  38.457439  51.427994  38.457439  44.604900  44.604900  200580054
                     2020-11-08  44.604900  54.786362  42.864254  53.749378  53.749378  174706009
                     2020-11-09  53.749374  58.364113  49.860130  54.886501  54.886501  386011348
                     2020-11-10  54.886505  70.309822  53.179035  67.001114  67.001114  433793206
                     2020-11-11  67.001114  68.778595  53.716755  54.699646  54.699646  354136841
                     2020-11-12  54.699623  59.618317  50.523102  51.632420  51.632420  244505562

Open       High        Low      Close  Adj Close  Volume
Asset Class Ticker Date                                                                     
bonds       SCHZ   2023-03-08  45.610001  45.700001  45.419998  45.480000  45.357838  624100
                   2023-03-09  45.520000  45.740002  45.509998  45.639999  45.517410  465500
                   2023-03-10  45.990002  46.290001  45.990002  46.180000  46.055962  624200
                   2023-03-13  46.610001  46.910000  46.419998  46.529999  46.405018  705600
                   2023-03-14  46.529999  46.529999  46.189999  46.290001  46.165665  535200
                   2023-03-15  46.720001  46.930000  46.529999  46.740002  46.614456  603600
                   2023-03-16  46.950001  46.959999  46.459999  46.520000  46.395046  992500
                   2023-03-17  46.709999  47.000000  46.709999  46.869999  46.744106  989600
                   2023-03-20  46.959999  46.959999  46.630001  46.700001  46.574562  709300
                   2023-03-21  46.500000  46.639999  46.480000  46.540001  46.414993  987500

In [14]:
data.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [15]:
# Check for 0s in the DataFrame
zero_count = np.count_nonzero(data.values == 0)
zero_count

0

In [16]:
# Check if any assset rows have value of 0
data[data==0].index.get_level_values(1).unique()

Index(['AAVE-USD', 'ALGO-USD', 'BAT-USD', 'BCH-USD', 'BTC-USD', 'DAI-USD',
       'ETH-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'MKR-USD', 'NEAR-USD',
       'PAXG-USD', 'SOL-USD', 'TRX-USD', 'USDT-USD ', 'WBTC-USD', 'SPY', 'QQQ',
       'IWM', 'VTI', 'VOO', 'VO', 'VB', 'VEA', 'VWO', 'XLF ', 'XLV', 'XLE',
       'XLY', 'XLC', 'XLK', 'XLI', 'XLP', 'XLB', 'XLU', 'XLRE', 'GLD', 'SLV',
       'USO', 'UNG', 'DBA', 'DBC', 'GSG', 'IAU ', 'PPLT', 'SIVR', 'MOO', 'NIB',
       'JO', 'JJG', 'WEAT', 'UGA', 'DBE', 'REMX', 'OIL', 'AGG', 'BND', 'TLT',
       'IEF', 'SHY', 'LQD', 'HYG', 'JNK', 'MUB', 'TIP', 'BNDX', 'EMB', 'VWOB',
       'PFF', 'BKLN', 'FLOT', 'GSY', 'SCHO', 'SCHR', 'SCHZ'],
      dtype='object', name='Ticker')

In [17]:
"""
Analize df for NaN and zero values. 
Print df containing the ticker name, zero values, NaN values, and dates for each ticker.
"""
def check_nan_and_zero_values(df):
    tickers = df.index.get_level_values('Ticker').unique()
    results = []

    for ticker in tickers:
        ticker_data = df.loc[pd.IndexSlice[:, ticker, :], :]
        nan_dates = ticker_data[ticker_data.isna().any(axis=1)].index.get_level_values('Date').tolist()
        zero_dates = ticker_data[(ticker_data == 0).any(axis=1)].index.get_level_values('Date').tolist()

        for date in nan_dates:
            results.append({"Ticker": ticker, "Value": "NaN", "Date": date})
        
        for date in zero_dates:
            results.append({"Ticker": ticker, "Value": "0", "Date": date})

    results_count_df = pd.DataFrame(results)
    return results_count_df



In [18]:
# Call the function to check for NaN and zero values in the data frame
results_count_df = check_nan_and_zero_values(data)
results_count_df.head(10)

""


In [19]:
#Remane column
data = data.rename(columns={"Adj Close": "Adj_Close"})

In [20]:
data.head()


Open       High        Low      Close  Adj_Close    Volume
Asset Class Ticker   Date                                                                       
crypto      AAVE-USD 2020-10-14  51.372898  57.853947  49.629528  51.316517  51.316517   3132405
                     2020-10-15  51.387611  52.080685  44.018326  44.019184  44.019184   5883146
                     2020-10-16  42.890606  43.906731  39.142956  39.495846  39.495846  52855205
                     2020-10-17  39.486191  43.077374  38.633614  41.366993  41.366993  43339687
                     2020-10-18  41.389599  41.749489  39.531586  40.746971  40.746971  40441163

### Step 3: Calculate returns for each asset
The `calculate_returns` function calculates the percentage returns for each asset, based on either the adjusted close or the close price. The function takes in a DataFrame containing the historical data for an asset and checks if the 'Adj Close' column is present. If it is, it calculates the returns using the 'Adj Close' column, and if not, it uses the 'Close' column.

In [21]:
# VAR 1 Regular returns
# Calculate returns for each asset
# def calculate_returns(df):
#     # If the 'Adj Close' column is present, use it for calculating returns
#     if 'Adj Close' in df.columns:
#         return df['Adj Close'].pct_change()
#     # If the 'Adj Close' column is not present, use the 'Close' column
#     else:
#         return  df['Close'].pct_change()
        

# VAR 2 Logarithmic returns returns
# Calculate logarithmic returns for each asset 
"""
Calculate the logarithmic returns of an asset.
If the 'Adj Close' column is present, use it for calculating log. returns.
Otherwise use the 'Close' column
"""

def calculate_log_returns(df):
    
    if 'Adj Close' in df.columns:
        return np.log(1 + df['Adj Close'].pct_change())    
    else:
        return np.log(1 + df['Close'].pct_change())
    

# data.groupby(['Asset Class', 'Ticker'], group_keys=False).apply(o_c_pct_change)
data["Returns"] = data.groupby(['Asset Class', 'Ticker'], group_keys=False).apply(calculate_log_returns)


In [22]:
display(data.head(20))
data.tail(20)

Open       High        Low      Close  Adj_Close    Volume   Returns
Asset Class Ticker   Date                                                                                 
crypto      AAVE-USD 2020-10-14  51.372898  57.853947  49.629528  51.316517  51.316517   3132405       NaN
                     2020-10-15  51.387611  52.080685  44.018326  44.019184  44.019184   5883146 -0.153387
                     2020-10-16  42.890606  43.906731  39.142956  39.495846  39.495846  52855205 -0.108430
                     2020-10-17  39.486191  43.077374  38.633614  41.366993  41.366993  43339687  0.046288
                     2020-10-18  41.389599  41.749489  39.531586  40.746971  40.746971  40441163 -0.015102
                     2020-10-19  40.836758  40.995186  35.951363  36.012318  36.012318  49289897 -0.123520
                     2020-10-20  36.043785  36.319069  32.101101  32.180016  32.180016  60632326 -0.112515
                     2020-10-21  32.253490  37.078201  32.048149  35.160358  35.160358  43831235  0.088574
                     2020-10-22  35.097488  39.856922  33.589462  38.031651  38.031651  54717366  0.078499
                     2020-10-23  37.730595  42.546658  36.924671  41.686638  41.686638  76081860  0.091762
                     2020-10-24  41.686638  42.439705  38.728821  39.455021  39.455021  57658170 -0.055019
                     2020-10-25  39.455021  40.928509  37.601200  37.904762  37.904762  43970452 -0.040085
                     2020-10-26  37.904762  39.163864  35.071968  36.033920  36.033920  50307223 -0.050616
                     2020-10-27  36.033913  38.869183  34.469635  34.871532  34.871532  58581308 -0.032790
                     2020-10-28  34.871536  35.564022  32.000057  32.394081  32.394081  60533867 -0.073695
                     2020-10-29  32.394081  33.155483  28.240772  29.098333  29.098333  66304786 -0.107295
                     2020-10-30  29.098341  31.980774  27.792461  31.217823  31.217823  77280724  0.070308
                     2020-10-31  31.217823  32.900162  29.610258  29.707518  29.707518  58631072 -0.049589
                     2020-11-01  29.707518  31.067060  28.494482  30.657276  30.657276  49568339  0.031470
                     2020-11-02  30.657282  33.923584  29.344650  29.731524  29.731524  87180146 -0.030662

Open       High        Low      Close  Adj_Close   Volume   Returns
Asset Class Ticker Date                                                                                
bonds       SCHZ   2023-02-22  45.840000  45.930000  45.790001  45.810001  45.573334  1119500  0.002623
                   2023-02-23  45.849998  46.009998  45.660000  46.009998  45.772301  1060800  0.004356
                   2023-02-24  45.759998  45.770000  45.639999  45.720001  45.483799   804500 -0.006323
                   2023-02-27  45.849998  45.880001  45.759998  45.799999  45.563385   388600  0.001748
                   2023-02-28  45.700001  45.849998  45.650002  45.840000  45.603180  1149500  0.000873
                   2023-03-01  45.599998  45.610001  45.419998  45.450001  45.327923   854700 -0.008544
                   2023-03-02  45.360001  45.360001  45.240002  45.330002  45.208244   805100 -0.002644
                   2023-03-03  45.560001  45.700001  45.470001  45.700001  45.577248   371200  0.008129
                   2023-03-06  45.750000  45.790001  45.560001  45.570000  45.447598   355300 -0.002849
                   2023-03-07  45.660000  45.669998  45.450001  45.500000  45.377785  1004500 -0.001537
                   2023-03-08  45.610001  45.700001  45.419998  45.480000  45.357838   624100 -0.000440
                   2023-03-09  45.520000  45.740002  45.509998  45.639999  45.517410   465500  0.003512
                   2023-03-10  45.990002  46.290001  45.990002  46.180000  46.055962   624200  0.011762
                   2023-03-13  46.610001  46.910000  46.419998  46.529999  46.405018   705600  0.007550
                   2023-03-14  46.529999  46.529999  46.189999  46.290001  46.165665   535200 -0.005171
                   2023-03-15  46.720001  46.930000  46.529999  46.740002  46.614456   603600  0.009674
                   2023-03-16  46.950001  46.959999  46.459999  46.520000  46.395046   992500 -0.004718
                   2023-03-17  46.709999  47.000000  46.709999  46.869999  46.744106   989600  0.007495
                   2023-03-20  46.959999  46.959999  46.630001  46.700001  46.574562   709300 -0.003634
                   2023-03-21  46.500000  46.639999  46.480000  46.540001  46.414993   987500 -0.003432

In [23]:
#Check for NaN values 
data.isna().sum()

Open          0
High          0
Low           0
Close         0
Adj_Close     0
Volume        0
Returns      76
dtype: int64

In [24]:
# Save the pulled data to CSV file
data.to_csv('../Capital_Allocation_Optimization/data.csv')

In [25]:
# Group the data by 'Asset Class' and 'Ticker', and apply the 'calculate_returns' function to each group.
# Assign the resulting series of calculated returns back to the original DataFrame as a new column named "Returns".
# data_with_returns = data.groupby(['Asset Class', 'Ticker'], group_keys=False).apply(calculate_returns)

# data_with_returns = data.groupby(['Asset Class', 'Ticker'], group_keys=False).apply(lambda x: calculate_returns(x, asset_class=x.name[0]))
# data_with_returns = data.groupby(['Asset Class', 'Ticker'], group_keys=False).apply(lambda x: calculate_returns(x, asset_class=x.name[0]))


In [26]:
# #Rewiev df
# display(data_with_returns.head(20))
# data_with_returns.tail(20)

### Step 4:Transpose the data DataFrame
Define a function `transpose_df` that takes in a pandas dataframe df and an asset_class string as input. The goal of the function is to transpose a subset of the dataframe based on the given asset_class and return the resulting pivoted dataframe with NaN values removed.

In [27]:
#Define function 
 # Selects data from the dataframe based on the given asset_class and Returns columns
def transpose_df(df, asset_class):
    df_data = df.loc[(asset_class),['Returns']].reset_index()

    # Pivot the dataframe with index as 'Ticker', columns as 'Date', and values as 'Returns'
    df_data_pivot = df_data.pivot(index='Ticker',columns='Date', values='Returns')

    # Drop columns containing any NaN values from the pivoted dataframe
    df_data_pivot = df_data_pivot.dropna(axis=1)
    return df_data_pivot



In [28]:

# Call transpose_df function and ranspose data subset for all asset classes, store in a new dataframe
df_crypto_data_transposed = transpose_df(data,'crypto')
df_stocks_data_transposed = transpose_df(data,'stocks')
df_commodities_data_transposed = transpose_df(data,'commodities')
df_bonds_data_transposed = transpose_df(data,'bonds')

In [29]:
# data.index

In [30]:
#Review transposed DataFrame for crypto assets 
df_crypto_data_transposed


Date,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,...,2023-03-12,2023-03-13,2023-03-14,2023-03-15,2023-03-16,2023-03-17,2023-03-18,2023-03-19,2023-03-20,2023-03-21
Ticker,,,,,,,,,,,,,,,,,,,,,
AAVE-USD,-0.153387,-0.108430,0.046288,-0.015102,-0.123520,-0.112515,0.088574,0.078499,0.091762,-0.055019,...,0.079546,0.042564,0.041045,-0.069440,0.014093,0.071859,-0.036130,0.022970,-0.044166,0.037100
ALGO-USD,-0.036625,-0.052106,0.043541,0.011925,-0.043294,-0.052727,0.048412,0.044629,-0.021238,-0.004266,...,0.067286,0.050767,0.044997,-0.083579,0.020757,0.082869,-0.052225,0.024458,-0.050270,0.066129
BAT-USD,-0.009601,-0.028967,0.036172,-0.003900,0.001644,-0.026485,0.029207,0.029983,-0.006866,0.005510,...,0.074826,0.057636,0.044676,-0.077281,0.019404,0.085176,-0.039028,0.037944,-0.039704,0.028248
BCH-USD,0.008650,-0.043232,-0.016130,0.014078,0.003587,-0.031870,0.062526,0.030319,0.014315,0.017628,...,0.056855,0.048577,0.033567,-0.062009,0.021323,0.072324,-0.032165,0.036023,-0.041412,0.029111
BTC-USD,0.005744,-0.015184,0.003173,0.010968,0.022276,0.014735,0.073384,0.011028,-0.002653,0.013558,...,0.071604,0.087784,0.022416,-0.015069,0.027388,0.090431,-0.016844,0.039013,-0.009728,0.014607
DAI-USD,0.003123,-0.002023,-0.001756,-0.000822,0.005126,-0.003665,-0.000085,0.000138,0.000329,-0.008978,...,0.019036,0.006358,0.000061,0.000188,0.000231,0.001078,-0.002316,0.000149,-0.001506,0.002242
ETH-USD,-0.005396,-0.030158,0.007147,0.025053,0.004542,-0.028834,0.060579,0.053571,-0.009730,0.006545,...,0.070110,0.055059,0.013711,-0.028175,0.012621,0.066468,-0.017347,0.013485,-0.028549,0.040343
LINK-USD,-0.019492,-0.011069,0.003100,0.021195,0.000512,-0.097133,0.089603,0.099866,0.018626,0.045619,...,0.059714,0.027893,0.029025,-0.062838,0.018666,0.079303,-0.041224,0.024091,-0.006341,0.048378
LTC-USD,-0.007170,-0.036996,-0.013488,0.011949,0.010016,-0.022027,0.119914,0.022775,0.017742,0.067493,...,0.090528,0.069061,0.031092,-0.098630,0.033774,0.078418,-0.029899,0.017350,-0.073858,0.053510


In [31]:
# data.loc[('crypto','SHIB-USD')]

In [32]:
#Review transposed DataFrame for stock assets 
df_stocks_data_transposed

Date,2020-10-15,2020-10-16,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-26,2020-10-27,2020-10-28,...,2023-03-08,2023-03-09,2023-03-10,2023-03-13,2023-03-14,2023-03-15,2023-03-16,2023-03-17,2023-03-20,2023-03-21
Ticker,,,,,,,,,,,,,,,,,,,,,
IWM,0.010252,-0.002461,-0.012021,0.002179,-0.008621,0.017107,0.005781,-0.021822,-0.008624,-0.031272,...,0.000482,-0.028208,-0.029254,-0.016424,0.018352,-0.016623,0.013559,-0.027363,0.012882,0.018281
QQQ,-0.006734,-0.005496,-0.016460,0.002147,-0.000774,-0.000035,0.001969,-0.015110,0.007778,-0.039767,...,0.004982,-0.017478,-0.014143,0.007389,0.022720,0.005232,0.026019,-0.004737,0.001996,0.014181
SPY,-0.001237,-0.000604,-0.015320,0.003998,-0.001895,0.005470,0.003389,-0.018653,-0.003453,-0.034777,...,0.001631,-0.018622,-0.014535,-0.001426,0.016395,-0.006274,0.017393,-0.015571,0.009570,0.013045
VB,0.006645,-0.002773,-0.011903,0.003903,-0.008066,0.016067,0.006199,-0.022082,-0.009553,-0.029857,...,0.000051,-0.026892,-0.030215,-0.018797,0.017019,-0.019380,0.013866,-0.023532,0.013561,0.018607
VEA,-0.011042,0.003133,-0.004341,0.005543,-0.003370,0.000964,0.005526,-0.018864,-0.007597,-0.029711,...,0.005185,-0.009944,-0.009814,-0.005750,0.011696,-0.030323,0.015392,-0.011639,0.011407,0.012880
VO,0.002960,-0.001183,-0.012450,0.003426,-0.002936,0.009051,0.008863,-0.020803,-0.006628,-0.030186,...,0.002332,-0.024378,-0.026653,-0.008220,0.014037,-0.016710,0.014026,-0.018048,0.012117,0.015367
VOO,-0.001190,-0.000846,-0.015225,0.003939,-0.001809,0.005480,0.003406,-0.018588,-0.003309,-0.035037,...,0.001556,-0.018475,-0.014444,-0.001693,0.016470,-0.006186,0.016963,-0.010998,0.008990,0.012996
VTI,0.000337,-0.001801,-0.014409,0.003479,-0.003422,0.007060,0.003285,-0.018319,-0.004040,-0.033696,...,0.001297,-0.019945,-0.016982,-0.004148,0.017665,-0.007738,0.017138,-0.012980,0.008672,0.014171
VWO,-0.008046,0.004031,-0.003134,0.010039,0.001996,0.000222,0.004640,-0.013092,0.003567,-0.025244,...,0.005243,-0.020630,-0.005608,-0.001023,0.000512,-0.016764,0.012663,-0.005149,0.001032,0.010517


In [33]:
#Review transposed DataFrame for commodities
df_commodities_data_transposed 

Date,2020-10-15,2020-10-16,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-26,2020-10-27,2020-10-28,...,2023-03-08,2023-03-09,2023-03-10,2023-03-13,2023-03-14,2023-03-15,2023-03-16,2023-03-17,2023-03-20,2023-03-21
Ticker,,,,,,,,,,,,,,,,,,,,,
DBA,-0.001344,-0.003368,0.004040,-0.000672,0.000672,0.002685,0.001340,-0.003352,-0.006739,-0.013615,...,-0.007446,-0.005997,0.004002,-0.005005,0.000000,-0.011100,0.007078,-0.001008,-0.000504,0.001009
DBC,0.002999,-0.006759,0.000753,0.011232,-0.008224,0.002999,-0.007513,-0.014432,0.006862,-0.026956,...,-0.005468,-0.012307,0.010618,-0.006357,-0.012837,-0.023530,0.003082,-0.003082,0.003960,0.003507
DBE,-0.003069,-0.008230,-0.005179,0.012384,-0.023872,0.011488,-0.015699,-0.023480,0.019252,-0.043297,...,-0.012632,-0.019012,0.013820,-0.014780,-0.027266,-0.040287,0.005126,-0.014937,0.007754,0.014823
GLD,0.003639,-0.003471,0.000505,0.004809,0.007503,-0.009849,-0.001063,-0.000504,0.002629,-0.016275,...,-0.000475,0.009801,0.021334,0.022689,-0.005808,0.007774,0.002018,0.028704,0.000381,-0.019056
GSG,0.005425,-0.005425,-0.003633,0.009058,-0.009968,0.003636,-0.008201,-0.016606,0.011101,-0.029881,...,-0.005431,-0.013958,0.010487,-0.010989,-0.017735,-0.028521,0.002627,-0.008959,0.006858,0.009940
IAU,0.003858,-0.003858,0.001104,0.004952,0.008199,-0.010947,-0.000551,-0.000551,0.002751,-0.017180,...,0.000291,0.008974,0.021666,0.023137,-0.006081,0.007458,0.002474,0.028682,0.000267,-0.018843
JJG,0.023408,-0.006265,0.001644,0.013322,0.006213,-0.004076,0.018567,-0.015008,-0.005178,-0.022574,...,-0.007063,-0.015994,0.009361,-0.009292,0.012511,0.003932,0.002884,0.001697,-0.000597,-0.012096
JO,0.000906,-0.029734,-0.000156,-0.013945,-0.011744,0.026404,-0.007742,0.013695,-0.004182,-0.017378,...,-0.031168,-0.009234,0.010843,0.011037,-0.019504,-0.014340,0.038526,-0.015092,0.008060,0.011403
MOO,-0.001152,0.001008,-0.008532,0.001161,0.000725,0.002317,0.002311,-0.015705,-0.004996,-0.023699,...,0.008665,-0.020717,-0.024432,-0.010132,0.015276,-0.029228,0.005270,-0.011896,0.014401,0.018065


In [34]:
#Review transposed DataFrame for bonds
df_bonds_data_transposed 

Date,2020-10-15,2020-10-16,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-26,2020-10-27,2020-10-28,...,2023-03-08,2023-03-09,2023-03-10,2023-03-13,2023-03-14,2023-03-15,2023-03-16,2023-03-17,2023-03-20,2023-03-21
Ticker,,,,,,,,,,,,,,,,,,,,,
AGG,-0.000848,-0.000509,-0.001188,-0.000850,-0.001191,-0.001789,0.001448,0.001361,0.002293,-0.001443,...,-0.001035,0.003616,0.011587,0.007920,-0.005884,0.009921,-0.003936,0.006251,-0.004028,-0.003032
BKLN,-0.001834,-0.001378,-0.000920,0.001379,-0.002299,-0.000461,0.000000,-0.003692,-0.000462,-0.003707,...,0.001428,-0.004291,-0.000956,-0.015422,0.006294,-0.007752,0.004368,-0.006315,-0.004396,0.008772
BND,-0.000567,-0.000795,-0.000909,-0.001706,-0.001139,-0.001711,0.001255,0.001481,0.002388,-0.001364,...,-0.000838,0.003348,0.011905,0.007950,-0.006300,0.010251,-0.004771,0.007215,-0.004350,-0.002729
BNDX,0.000856,0.000855,-0.000342,-0.001027,-0.001885,-0.001373,0.001030,0.000858,0.001884,-0.000171,...,0.001676,0.001046,0.009778,0.012755,-0.005946,0.007171,-0.004912,0.007155,-0.000815,-0.004905
EMB,-0.004011,0.001517,-0.005814,0.000538,-0.005123,-0.004697,0.004697,-0.003611,0.005951,-0.008578,...,-0.004740,0.000237,0.009338,-0.001648,-0.002714,0.003656,-0.004247,-0.004503,0.000119,0.007100
FLOT,-0.000395,0.000197,-0.000197,0.000197,0.000197,-0.000197,0.000197,-0.000197,0.000000,0.000000,...,0.000394,-0.000394,-0.000197,-0.015911,0.003402,-0.010848,0.008247,-0.000802,0.001002,0.007582
GSY,0.000198,-0.000198,-0.000396,0.000000,0.000000,-0.000198,0.000198,0.000198,0.000198,-0.000198,...,-0.000202,0.000403,0.001410,0.002011,-0.001005,0.000402,-0.000402,0.000402,-0.003020,-0.000605
HYG,-0.000708,-0.001181,-0.003551,0.003551,-0.000118,0.001771,0.001179,-0.008876,0.000475,-0.008352,...,-0.004729,-0.005568,0.000136,-0.005462,0.007230,-0.003268,0.006524,-0.005979,-0.002730,0.010874
IEF,-0.001071,-0.000577,-0.001651,-0.002150,-0.001326,-0.003488,0.001413,0.002655,0.002234,-0.000083,...,-0.000316,0.006193,0.018248,0.012051,-0.009486,0.014346,-0.009031,0.011755,-0.004038,-0.008737


In [35]:
#Prints the sum of all NaN values in each of the dataframes.
print(df_crypto_data_transposed.isna().sum().sum())
print(df_stocks_data_transposed.isna().sum().sum())
print(df_commodities_data_transposed.isna().sum().sum())
print(df_bonds_data_transposed.isna().sum().sum())

0
0
0
0


### Step 5: Perform Principal Component Analysis (PCA)
Apply Principal Component Analysis (PCA) on the transposed data of each asset class to reduce the dimensionality of the data while preserving most of the variability in the data.
We replace infinite values with NaN before filling missing values with the mean of the corresponding column. This ensures that no infinite values are present in the data before it is passed to the PCA function.

In [36]:

# def pca_function(df):
#     pca = PCA(n_components=0.9) # Choose the number of principal components to explain 90% of the variance
#     pca.fit(df)
#     results = pca.transform(df)
#     results_df = pd.DataFrame(results,index = df.index)
#     return results_df

# #Define function for PCA
# def perform_pca(df):
#     """
#     Perform PCA on a given dataframe and return a dataframe of principal components.

#     Parameters:
#     df (pandas.DataFrame): The dataframe to perform PCA on.

#     Returns:
#     pandas.DataFrame: A dataframe of principal components.
#     """
#     # Choose the number of principal components to explain 90% of the variance
#     pca = PCA(n_components=0.9)

#     # Fit the PCA model to the data
#     pca.fit(df)

#     # Transform the data into principal components
#     results = pca.transform(df)

#     # Create a dataframe from the principal components
#     results_df = pd.DataFrame(results, index=df.index)

#     # Return the dataframe of principal components
#     return results_df

# Define function for PCA
def perform_pca(df):
    """
    Perform PCA on a given dataframe and return a dataframe of principal components.

    Parameters:
    df (pandas.DataFrame): The dataframe to perform PCA on.

    Returns:
    pandas.DataFrame: A dataframe of principal components.
    """
    # Replace NaN and infinite values with column means
    # df_clean = df.replace([np.inf, -np.inf], np.nan).fillna(df.mean())

    # Choose the number of principal components to explain 90% of the variance
    pca = PCA(n_components=0.9)

    # Fit the PCA model to the cleaned data
    pca.fit(df)

    # Transform the data into principal components
    results = pca.transform(df)

    # Create a dataframe from the principal components
    results_df = pd.DataFrame(results, index=df.index)

    # Return the dataframe of principal components
    return results_df



In [37]:
# # Replace missing values with the mean value of the corresponding column for all 4 asset classes
# df_crypto_data_transposed_clean = df_crypto_data_transposed.fillna(df_crypto_data_transposed.mean())
# df_stocks_data_transposed_clean = df_stocks_data_transposed.fillna(df_stocks_data_transposed.mean())
# df_commodities_data_transposed_clean = df_commodities_data_transposed.fillna(df_commodities_data_transposed.mean())
# df_bonds_data_transposed_clean = df_bonds_data_transposed.fillna(df_bonds_data_transposed.mean())

# Apply PCA function for all asset classes
crypto_pca = perform_pca(df_crypto_data_transposed)
stocks_pca = perform_pca(df_stocks_data_transposed)
commodities_pca = perform_pca(df_commodities_data_transposed)
bonds_pca = perform_pca(df_bonds_data_transposed)

/Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['Timestamp']. An error will be raised in 1.2.
  FutureWarning,
/Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['Timestamp']. An error will be raised in 1.2.
  FutureWarning,
/Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['Timestamp']. An error will be raised in 1.2.
  FutureWarning,
/Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['Timestamp']. A

In [38]:
crypto_pca

,0,1,2,3,4,5,6,7,8
Ticker,,,,,,,,,
AAVE-USD,-0.570583,0.045162,0.255590,-0.076656,0.783203,0.280939,-0.676692,-0.141890,0.147873
ALGO-USD,-0.368980,0.032286,-0.302564,-0.171440,-0.290116,0.471319,-0.097419,0.733359,-0.199390
BAT-USD,-0.278138,-0.326183,-0.187603,-0.237274,-0.338506,0.717847,0.269857,-0.586403,-0.039578
BCH-USD,-0.068406,-0.362772,-0.212868,-0.295586,-0.135632,-0.466894,-0.004749,-0.049279,0.085336
BTC-USD,0.358404,-0.077265,-0.050898,-0.050554,-0.027205,-0.181520,0.000767,-0.104665,-0.156667
DAI-USD,1.107127,0.220279,0.081335,0.206858,0.032129,0.105692,-0.019474,0.010108,-0.022610
ETH-USD,-0.076685,-0.131541,0.059386,-0.131362,0.142088,-0.185418,-0.010006,-0.025210,-0.112901
LINK-USD,-0.403722,-0.262198,-0.143312,-0.181907,-0.000243,-0.054175,-0.210620,0.057319,-0.309920
LTC-USD,-0.085793,-0.294933,-0.162413,-0.240789,-0.107033,-0.397762,-0.134817,-0.069799,-0.142440


In [39]:
stocks_pca

,0,1,2,3,4,5,6,7
Ticker,,,,,,,,
IWM,0.012886,-0.101292,-0.090697,0.020360,0.056707,0.008966,-0.011206,0.072932
QQQ,-0.156376,-0.088899,0.054146,-0.043808,-0.002318,-0.018999,-0.008897,-0.013738
SPY,-0.036351,-0.004146,0.009576,-0.025161,-0.022890,-0.013464,-0.007912,-0.007474
VB,0.010293,-0.075654,-0.077632,0.017578,0.040089,0.008298,-0.010274,0.044894
VEA,0.021215,0.014856,0.029539,0.085787,-0.005862,0.003942,-0.004957,-0.034707
VO,-0.020809,-0.026702,-0.036770,-0.001473,0.019454,-0.000538,-0.015299,0.011775
VOO,-0.037568,-0.003400,0.010707,-0.023787,-0.023081,-0.013531,-0.006489,-0.006143
VTI,-0.037163,-0.023851,-0.000369,-0.015530,-0.009055,-0.011083,-0.010095,0.005463
VWO,-0.000883,-0.009515,0.117407,0.177681,0.048815,-0.000349,0.009203,0.000632


In [40]:
commodities_pca

,0,1,2,3,4,5,6
Ticker,,,,,,,
DBA,-0.039407,-0.097435,-0.134896,0.064394,0.003250,0.028167,-0.017601
DBC,-0.021652,0.123272,-0.020835,-0.015946,-0.006957,0.001170,-0.009583
DBE,0.003098,0.291482,0.026495,-0.011596,0.021545,-0.000267,-0.009479
GLD,-0.057061,-0.168668,0.004464,-0.100962,0.056514,0.027809,-0.042175
GSG,-0.019705,0.178463,-0.021293,-0.018129,-0.008966,-0.008214,-0.005558
IAU,-0.057116,-0.168714,0.004331,-0.100295,0.055907,0.028169,-0.042617
JJG,-0.055212,-0.091587,-0.260542,0.012931,-0.161407,-0.016780,-0.011287
JO,-0.061607,-0.130252,-0.157016,0.293037,0.292952,-0.214596,-0.003002
MOO,-0.020704,-0.104750,0.082499,0.073959,-0.028945,0.070369,-0.024779


In [41]:
bonds_pca

,0,1,2,3,4
Ticker,,,,,
AGG,0.023421,-0.020149,-0.001272,0.004559,0.003085
BKLN,-0.068651,0.002684,0.013647,0.004495,-0.027401
BND,0.024377,-0.020996,-0.001416,0.003688,0.002496
BNDX,0.002123,-0.036523,-0.002504,0.001497,-0.000624
EMB,0.011134,0.097262,-0.055784,-0.023916,-0.004744
FLOT,-0.062849,-0.050300,0.006166,-0.015301,-0.018688
GSY,-0.053319,-0.057636,-0.003072,-0.011358,-0.012646
HYG,-0.044384,0.068331,0.004768,0.043569,0.001371
IEF,0.055183,-0.027486,-0.002897,0.006921,0.010824


In [42]:
# Define number of clusters
crypto_num_clusters = len(crypto_tickers)
stocks_num_clusters = len(stocks_tickers)
commodities_num_clusters = len(commodities_tickers)
bonds_num_clusters = len(bonds_tickers)

In [43]:
crypto_num_clusters 

17

In [44]:
stocks_num_clusters 

20

In [45]:

commodities_num_clusters 

19

In [46]:

bonds_num_clusters 

20

### Step : Write function to efine clusters, pick the best asset from eachj cluster based on sharpe ratio.
Here is a step by step description of the function: 
1. The function takes in four parameters - `df`, ` df_transpose_returns`, and `num_clusters` - which represent the PCA-transformed data, the transpose of the returns data, and the number of clusters to be created.
2. Creates an empty list called `'inertias'` to store the inertias of each KMeans object.
3. It iterates over the numbers 1 to the value of `'num_clusters'` (inclusive), and for each iteration, it:
   a) Instantiates a KMeans object with the current iteration number of clusters and a               random state of 42.                        
   b) Fits the KMeans object to the DataFrame (df) and calculates the inertia.
   c) Appends the inertia of the KMeans object to the `'inertias'` list.
4. Finds the number of clusters that produce the lowest inertia by finding the index of the minimum value in the `'inertias'`list and adding 1 to it.
5. Calculates the mean inertia of the KMeans objects.
6. Iterates over the numbers 1 to the value of 'num_clusters' (inclusive), and for each iteration, it:
    a) Checks if the inertia of the KMeans object is less than or equal to the mean inertia.
    b) If it is, it sets the number of clusters to the current iteration number and breaks the loop.
7. The function instantiates a KMeans object with the best number of clusters found in step 4.
8. Fits the KMeans object to the DataFrame.
9. Creates a DataFrame of cluster labels.
10. Calculates the Sharpe ratio of the asset class by applying a lambda function to the transpose of the returns data `(df_transpose_returns)`.
11. Combines the cluster labels with the Sharpe ratios and renames the first column to `'Sharpe_Ratio'`.
12. Gets the best Sharpe ratios for each cluster by grouping the DataFrame by `'Clusters'` and getting the maximum value of `'Sharpe_Ratio'` for each group.
13. Gets the tickers for the assets with the best Sharpe ratios by selecting the rows in the DataFrame where the `'Sharpe_Ratio'` column is in the list of best Sharpe ratios found in step 12 and getting their index values as a list.
14. Returns the DataFrame of cluster labels, the DataFrame of tickers with the best Sharpe ratios, and the list of tickers as three separate values.


In [47]:
# Define clusters, pick the best asset from eachj cluster based on sharpe ratio
def define_clusters(df, df_transpose_returns, num_clusters):  
    # Define an empty list to store the inertias of each KMeans object
    inertias = []
    
    # Iterate over the numbers 1 to num_clusters (inclusive)
    for n in range(1, num_clusters +1):
        # Instantiate a KMeans object with n clusters
        kmeans = KMeans(n_clusters=n, random_state=42)
        # Fit the KMeans object to the DataFrame and calculate the inertia
        kmeans.fit(df)
        # Append the inertia of the KMeans object to the list
        inertias.append(kmeans.inertia_)
    
    # Find the number of clusters that produces the lowest inertia
    best_num_clusters = np.argmin(inertias) + 1
    # Calculate the mean inertia of the KMeans objects
    mean_inertia = np.mean(inertias)
    
    # Iterate over the numbers 1 to num_clusters (inclusive)
    for k in range(1, num_clusters +1 ):
        # If the inertia of the KMeans object is less than or equal to the mean inertia
        if inertias[k-1] <= mean_inertia:
            # Set the number of clusters to k and break the loop
            best_num_clusters = k
            break
    
    # Instantiate a KMeans object with the best number of clusters
    kmeans_final = KMeans(n_clusters=best_num_clusters, random_state=42)
    # Fit the KMeans object to the DataFrame
    kmeans_final.fit(df)   
    # Create a DataFrame of cluster labels
    final_df = pd.DataFrame(kmeans_final.predict(df), index=df.index, columns=['Clusters'])
    
    # Calculate the Sharpe ratio of the asset class
    sharpe_ratio = df_transpose_returns.apply(lambda x: x.mean() / x.std(), axis=1)
    
    # Combine the cluster labels with the Sharpe ratios
    sharpe_cluster = pd.concat([final_df, sharpe_ratio], axis=1).rename(columns={0: 'Sharpe_Ratio'})
    
    # Get the best Sharpe ratios for each cluster
    best_cluster_sharpes = sharpe_cluster.groupby('Clusters').max()['Sharpe_Ratio'].to_list()
    
    # Get the tickers for the assets with the best Sharpe ratios
    best_cluster_sharpes_df = sharpe_cluster.loc[sharpe_cluster['Sharpe_Ratio'].isin(best_cluster_sharpes)]
    list_best_tickers = best_cluster_sharpes_df.index.to_list()
    
    # Return the DataFrame of cluster labels, the DataFrame of tickers with the best Sharpe ratios, and the list of tickers
    return sharpe_cluster, best_cluster_sharpes_df, list_best_tickers, 

In [48]:
# # VAR 1 Call the function
crypto_sharpe_ratio, best_cluster_crypto, best_crypto_ticekrs = define_clusters(crypto_pca, df_crypto_data_transposed, crypto_num_clusters)
stocks_sharpe_ratio, best_cluster_stocks, best_stocks_ticekrs = define_clusters(stocks_pca,  df_stocks_data_transposed, stocks_num_clusters)
commodities_sharpe_ratio, best_cluster_commodities, best_commodities_ticekrs = define_clusters(commodities_pca, df_commodities_data_transposed, commodities_num_clusters)
bonds_sharpe_ratio, best_cluster_bonds, best_bonds_ticekrs = define_clusters(bonds_pca, df_bonds_data_transposed, bonds_num_clusters)


In [49]:
best_cluster_crypto

,Clusters,Sharpe_Ratio
Ticker,,
AAVE-USD,0,0.007136
ETH-USD,2,0.035994
MATIC-USD,3,0.061082
MKR-USD,6,0.004056
NEAR-USD,5,0.008232
PAXG-USD,1,0.003490
SOL-USD,4,0.034282


In [50]:
best_cluster_stocks

,Clusters,Sharpe_Ratio
Ticker,,
VB,3,0.013592
VOO,1,0.019366
VWO,5,-0.018204
XLE,0,0.073293
XLF,4,0.029483
XLK,6,0.016944
XLU,7,0.005028
XLV,2,0.028377


In [51]:
best_cluster_commodities

,Clusters,Sharpe_Ratio
Ticker,,
DBA,5,0.052786
JJG,0,0.044262
PPLT,3,0.009417
REMX,4,0.037281
UGA,2,0.068994
UNG,1,-0.016490


In [52]:
best_cluster_bonds

,Clusters,Sharpe_Ratio
Ticker,,
FLOT,0,-0.009948
HYG,3,-0.038730
PFF,5,-0.043881
TIP,1,-0.052549
TLT,2,-0.065528
VWOB,4,-0.063037


When building a diversified portfolio, the goal is to minimize unsystematic risk through diversification across various assets, sectors, or asset classes. In this context, the Treynor Ratio can be a more suitable metric to evaluate risk-adjusted returns, as it accounts for systematic risk (also known as market risk or non-diversifiable risk) rather than total risk.

The Treynor Ratio is calculated as follows:
Treynor Ratio = (Average Portfolio Return - Risk-Free Rate) / Portfolio Beta
Here, the Portfolio Beta measures an investment's sensitivity to market movements, representing the systematic risk of the investment. A higher Treynor Ratio indicates better risk-adjusted performance, as the investment generates a higher return per unit of systematic risk.

 Pull data for benchmarks and the daily yield of a short-term U.S. government bond using yfinance package.

In [53]:
# Define benchmark tickers for each asset class
benchmark_tickers = {
    'crypto': 'BTC-USD',  # Bitcoin as a benchmark for cryptocurrencies
    'stocks': 'SPY',      # S&P 500 ETF (SPY) as a benchmark for stocks
    'commodities': 'GSG', # S&P GSCI Commodity Index ETF (GSG) as a benchmark for commodities
    'bonds': 'AGG'        # iShares Core U.S. Aggregate Bond ETF (AGG) as a benchmark for bonds
}

# Define the ticker for a short-term U.S. government bond (1-3 month T-bill)
risk_free_rate_ticker = '^IRX'  # 13 Week Treasury Bill as risk-free rate


# Download historical data for benchmark tickers
benchmark_data = yf.download(list(benchmark_tickers.values()), start=selected_start_date, end=selected_end_date)['Adj Close']

# Fill NaN values with the previous data point (forward fill)
benchmark_data.fillna(method='ffill', inplace=True)

# Download historical data for the risk-free rate (13 Week Treasury Bill)
risk_free_rate_data = yf.download(risk_free_rate_ticker, start=selected_start_date, end=selected_end_date)['Adj Close']

# Fill NaN values with the previous data point (forward fill)
risk_free_rate_data.fillna(method='ffill', inplace=True)

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


In [54]:
#!!! Devide crypto and other assets(ETF's don't trade on weekends

In [55]:
 benchmark_data.head(14)

,AGG,BTC-USD,GSG,SPY
Date,,,,
2020-10-14,112.047585,11429.506836,11.03,335.158478
2020-10-15,111.952660,11495.349609,11.09,334.744293
2020-10-16,111.895706,11322.123047,11.03,334.541992
2020-10-17,111.895706,11358.101562,11.03,334.541992
2020-10-18,111.895706,11483.359375,11.03,334.541992
2020-10-19,111.762848,11742.037109,10.99,329.455780
2020-10-20,111.667908,11916.334961,11.09,330.775543
2020-10-21,111.535027,12823.689453,10.98,330.149353
2020-10-22,111.335716,12965.891602,11.02,331.960358


In [56]:
#Check for 0 values 
benchmark_data.isna().sum()


AGG        0
BTC-USD    0
GSG        0
SPY        0
dtype: int64

In [57]:
risk_free_rate_data.head(14)

Date
2020-10-14    0.100
2020-10-15    0.095
2020-10-16    0.090
2020-10-19    0.090
2020-10-20    0.095
2020-10-21    0.090
2020-10-22    0.085
2020-10-23    0.083
2020-10-26    0.080
2020-10-27    0.090
2020-10-28    0.090
2020-10-29    0.083
2020-10-30    0.083
2020-11-02    0.083
Name: Adj Close, dtype: float64

In [58]:
#Check for 0 values 
risk_free_rate_data.isna().sum()

0

The next step is to calculate daily returns for both the benchmark_data and risk_free_rate_data. We'll also calculate the daily excess returns for each asset class, subtracting the risk-free rate from the asset returns.

Excess returns, also known as risk-adjusted returns, represent the difference between the actual returns of an investment and the returns of a risk-free investment (usually a short-term government bond). In other words, excess returns measure the performance of an investment after accounting for the risk-free rate, which is the return an investor would expect from an investment with zero risk.

The excess return helps to evaluate the performance of an investment by comparing it to a benchmark, such as a risk-free investment or a market index, and determining whether the investment has outperformed or underperformed the benchmark. This allows investors to determine the additional return they are receiving for taking on additional risk, as higher-risk investments are generally expected to yield higher returns.

In [59]:
# Calculate daily log returns for benchmark_data of each 
crypto_benchmark_log_returns = np.log(1 + benchmark_data['BTC-USD'].pct_change())

In [60]:
#Check for 0 values 
crypto_benchmark_log_returns.isna().sum()

1

In [61]:
crypto_benchmark_log_returns.head(14)

Date
2020-10-14         NaN
2020-10-15    0.005744
2020-10-16   -0.015184
2020-10-17    0.003173
2020-10-18    0.010968
2020-10-19    0.022276
2020-10-20    0.014735
2020-10-21    0.073384
2020-10-22    0.011028
2020-10-23   -0.002653
2020-10-24    0.013558
2020-10-25   -0.005883
2020-10-26    0.003377
2020-10-27    0.043328
Freq: D, Name: BTC-USD, dtype: float64

In [62]:
stocks_benchmark_log_returns = np.log(1 + benchmark_data['SPY'].pct_change())

In [63]:
#Check for 0 values 
stocks_benchmark_log_returns.isna().sum()

1

In [64]:
stocks_benchmark_log_returns.head(14)

Date
2020-10-14         NaN
2020-10-15   -0.001237
2020-10-16   -0.000605
2020-10-17    0.000000
2020-10-18    0.000000
2020-10-19   -0.015320
2020-10-20    0.003998
2020-10-21   -0.001895
2020-10-22    0.005470
2020-10-23    0.003389
2020-10-24    0.000000
2020-10-25    0.000000
2020-10-26   -0.018653
2020-10-27   -0.003453
Freq: D, Name: SPY, dtype: float64

In [65]:
commodities_benchmark_log_returns = np.log(1 + benchmark_data['GSG'].pct_change())

In [66]:
#Check for 0 values 
commodities_benchmark_log_returns.isna().sum()

1

In [67]:
commodities_benchmark_log_returns.head(14)

Date
2020-10-14         NaN
2020-10-15    0.005425
2020-10-16   -0.005425
2020-10-17    0.000000
2020-10-18    0.000000
2020-10-19   -0.003633
2020-10-20    0.009058
2020-10-21   -0.009968
2020-10-22    0.003636
2020-10-23   -0.008201
2020-10-24    0.000000
2020-10-25    0.000000
2020-10-26   -0.016606
2020-10-27    0.011101
Freq: D, Name: GSG, dtype: float64

In [68]:
bonds_benchmark_log_returns = np.log(1 + benchmark_data['AGG'].pct_change())

In [69]:
#Check for 0 values 
bonds_benchmark_log_returns.isna().sum()

1

In [70]:
bonds_benchmark_log_returns.head(14)

Date
2020-10-14         NaN
2020-10-15   -0.000848
2020-10-16   -0.000509
2020-10-17    0.000000
2020-10-18    0.000000
2020-10-19   -0.001188
2020-10-20   -0.000850
2020-10-21   -0.001191
2020-10-22   -0.001789
2020-10-23    0.001448
2020-10-24    0.000000
2020-10-25    0.000000
2020-10-26    0.001361
2020-10-27    0.002293
Freq: D, Name: AGG, dtype: float64

In [71]:
#!!! Resk free rate log returns doesn't have data for weekends. When we calculating exess returns we need to substract risk_free_rate_log_returns!!!

In [72]:
# # Calculate excess log returns for each asset class by subtracting the risk-free rate daily log returns from asset daily log returns
# crypto_excess_log_returns = df_crypto_data_transposed.T.subtract(risk_free_rate_log_returns, axis=0)
# excess_log_returns_stocks = df_stocks_data_transposed.T.subtract(risk_free_rate_log_returns, axis=0)
# excess_log_returns_commodities = df_commodities_data_transposed.T.subtract(risk_free_rate_log_returns, axis=0)
# excess_log_returns_bonds = df_bonds_data_transposed.T.subtract(risk_free_rate_log_returns, axis=0)

In [73]:
# benchmark_log_returns.head(10)

In [74]:
# risk_free_rate_log_returns.head(10)

In [75]:
# excess_log_returns_crypto.head(10)

In [76]:
# excess_log_returns_stocks.head(10)

In [77]:
#Select the data for the best tickers in each asset class and transpose the resulting dataframes.
crypto_results = df_crypto_data_transposed.loc[best_crypto_ticekrs].T
stocks_results = df_stocks_data_transposed.loc[best_stocks_ticekrs].T
commodities_results = df_commodities_data_transposed.loc[best_commodities_ticekrs].T
bonds_results = df_bonds_data_transposed.loc[best_bonds_ticekrs].T

In [78]:
crypto_results.head(14)

Ticker,AAVE-USD,ETH-USD,MATIC-USD,MKR-USD,NEAR-USD,PAXG-USD,SOL-USD
Date,,,,,,,
2020-10-15,-0.153387,-0.005396,-0.002614,-0.008630,-0.052145,0.001917,-0.021043
2020-10-16,-0.108430,-0.030158,-0.056050,-0.020481,-0.319769,-0.002297,-0.053026
2020-10-17,0.046288,0.007147,0.029643,0.020188,-0.011534,-0.003785,0.024113
2020-10-18,-0.015102,0.025053,0.008624,0.029280,0.075685,0.001764,0.005627
2020-10-19,-0.123520,0.004542,-0.041224,-0.002855,-0.085346,0.001931,-0.073148
2020-10-20,-0.112515,-0.028834,-0.036000,-0.038822,-0.168786,0.000810,-0.085900
2020-10-21,0.088574,0.060579,0.046924,0.033010,0.036804,0.011460,0.000903
2020-10-22,0.078499,0.053571,0.060456,0.040550,0.065964,-0.008752,0.033947
2020-10-23,0.091762,-0.009730,-0.021137,0.001254,-0.028327,0.001672,0.002238


In [79]:
stocks_results.head(14)

Ticker,VB,VOO,VWO,XLE,XLF,XLK,XLU,XLV
Date,,,,,,,,
2020-10-15,0.006645,-0.001190,-0.008046,0.012050,0.007638,-0.004518,-0.000632,-0.007273
2020-10-16,-0.002773,-0.000846,0.004031,-0.022922,0.000801,-0.002886,0.010682,0.009871
2020-10-19,-0.011903,-0.015225,-0.003134,-0.021427,-0.015323,-0.018499,-0.008473,-0.015972
2020-10-20,0.003903,0.003939,0.010039,0.011775,0.008094,0.003442,0.005970,0.001317
2020-10-21,-0.008066,-0.001809,0.001996,-0.019248,-0.005254,-0.001510,-0.002666,-0.004996
2020-10-22,0.016067,0.005480,0.000222,0.040426,0.019660,-0.004712,0.014036,0.015007
2020-10-23,0.006199,0.003406,0.004640,-0.004924,0.003173,-0.001097,0.002784,0.003717
2020-10-26,-0.022082,-0.018588,-0.013092,-0.036526,-0.022833,-0.021938,-0.000309,-0.011192
2020-10-27,-0.009553,-0.003309,0.003567,-0.013054,-0.018402,0.004993,-0.000773,-0.006870


In [80]:
commodities_results.head(14)

Ticker,DBA,JJG,PPLT,REMX,UGA,UNG
Date,,,,,,
2020-10-15,-0.001344,0.023408,0.007156,-0.006091,-0.011400,0.049638
2020-10-16,-0.003368,-0.006265,-0.002832,-0.005145,-0.016081,-0.008918
2020-10-19,0.004040,0.001644,-0.009290,0.009778,-0.007117,0.016956
2020-10-20,-0.000672,0.013322,0.020812,0.008236,0.022201,-0.008846
2020-10-21,0.000672,0.006213,0.017518,0.001206,-0.032457,0.020784
2020-10-22,0.002685,-0.004076,-0.007573,-0.005073,0.014330,-0.019977
2020-10-23,0.001340,0.018567,0.025143,-0.001454,-0.018462,-0.023684
2020-10-26,-0.003352,-0.015008,-0.032045,-0.014659,-0.026752,0.016394
2020-10-27,-0.006739,-0.005178,0.005453,0.009797,0.026234,0.018526


In [81]:
bonds_results.head(14)

Ticker,FLOT,HYG,PFF,TIP,TLT,VWOB
Date,,,,,,
2020-10-15,-0.000395,-0.000708,0.000271,-0.000317,-0.001729,-0.004141
2020-10-16,0.000197,-0.001181,-0.003528,0.000396,-0.002784,0.000754
2020-10-19,-0.000197,-0.003551,-0.004633,-0.002377,-0.003787,-0.004281
2020-10-20,0.000197,0.003551,0.006263,-0.000873,-0.009562,0.000757
2020-10-21,0.000197,-0.000118,-0.007903,-0.000318,-0.003460,-0.004169
2020-10-22,-0.000197,0.001771,0.007903,-0.001510,-0.010451,-0.004569
2020-10-23,0.000197,0.001179,0.006224,0.001193,0.006094,0.004189
2020-10-26,-0.000197,-0.008876,-0.008670,0.000794,0.009260,-0.002917
2020-10-27,0.000000,0.000475,0.000544,0.001983,0.006687,0.005321


In [82]:
# Define tickers selected for each asset class
crypto_selected_tickers = crypto_results.columns.tolist()
stocks_selected_tickers = stocks_results.columns.tolist()
commodities_selected_tickers = commodities_results.columns.tolist()
bonds_selected_tickers = bonds_results.columns.tolist()


In [83]:
 crypto_selected_tickers

['AAVE-USD',
 'ETH-USD',
 'MATIC-USD',
 'MKR-USD',
 'NEAR-USD',
 'PAXG-USD',
 'SOL-USD']

In [84]:
stocks_selected_tickers

['VB', 'VOO', 'VWO', 'XLE', 'XLF ', 'XLK', 'XLU', 'XLV']

In [85]:
commodities_selected_tickers

['DBA', 'JJG', 'PPLT', 'REMX', 'UGA', 'UNG']

In [86]:
bonds_selected_tickers

['FLOT', 'HYG', 'PFF', 'TIP', 'TLT', 'VWOB']

In [87]:
# Calculate the number of tickers selected for each asset class
crypto_num_selected_tickers = len(crypto_selected_tickers)
stocks_num_selected_tickers = len(stocks_selected_tickers)
commodities_num_selected_tickers = len(commodities_selected_tickers)
bonds_num_selected_tickers = len(bonds_selected_tickers)

In [88]:
crypto_num_selected_tickers

7

In [89]:
stocks_num_selected_tickers

8

In [90]:
commodities_num_selected_tickers

6

In [91]:
bonds_num_selected_tickers

6

### Step : Perform Monte carlo simulation.
The num_of_portfolios constant defines how many portfolios will be generated by the Monte Carlo simulation. The `CRYPTO_TRADING_DAYS` constant is the number of trading days in a year for the cryptocurrencies asset class(365 days), and the `ETFS_TRADING_DAYS` constant is the number of trading days in a year for the stocks, commodities, and bonds asset classes(252 days). These constants are used in the monte_carlo_simulation function to calculate expected returns, volatilities, and Sharpe ratios for each portfolio.
To run Monte Carlo simulation we also defining `num_of_portfolios` variable that is the number of simulations that will be generated.

Here are the step by steo explanation of the `monte_carlo_simulation` function :

1. The function takes in three parameters:
* `num_of_portfolios` - an integer specifying the number of portfolios to be generated by the Monte Carlo simulation
* `num_selected_tickers` - an integer specifying the number of selected tickers for the asset class
* `results` - a Pandas DataFrame containing results for the asset class
2. It performs a Monte Carlo simulation to generate portfolios and calculate their expected returns, volatilities, and Sharpe ratios for a single asset class.
3. The function first checks if the results parameter is for crypto assets or ETFs, and sets the trading days constant accordingly.
4. Then it initializes four NumPy arrays to store weights, returns, volatilities, and Sharpe ratios for each portfolio.
5. Loops through the number of portfolios specified, and for each portfolio:
* Calculates random weights for the selected tickers and normalizes them
* Stores the weights in the `all_weights` array
* Calculates the expected returns for the portfolio and stores them in the `ret_arr` array
* Calculates the volatility for the portfolio and stores it in the `vol_arr` array
* Calculates the Sharpe ratio for the portfolio and stores it in the `sharpe_ratio_arr` array
6. Returns a tuple of the four NumPy arrays containing the weights, returns, volatilities, and Sharpe ratios for all generated portfolios.

In [92]:
# Define trading days constant
CRYPTO_TRADING_DAYS = 365
ETFS_TRADING_DAYS = 252

In [93]:
# Prompt user to choose number of simulations for Streamlit !!!
#num_of_portfolios = st.slider("Choose number of portfolios simulated:", min_value=500, max_value=5000, step=500)

In [94]:
#Define number of simulations(portfolios generated by Monte Carlo)
num_of_portfolios = 3000

In [95]:
def monte_carlo_simulation(num_of_portfolios, num_selected_tickers, results):
    """
    This function performs a Monte Carlo simulation for generating portfolios and calculating their
    expected returns, volatilities, and Sharpe ratios for a single asset class.
    
    Parameters:
    - num_of_portfolios (int): Number of portfolios to be generated by the Monte Carlo simulation.
    - num_selected_tickers (int): Number of selected tickers for the asset class.
    - results (DataFrame): DataFrame containing results for the asset class.
    
    Returns:
    - Tuple of 4 NumPy arrays: weights, returns, volatilities, and Sharpe ratios for the asset class.
    """
    if results is crypto_results or 'Crypto' in results.columns or 'Crypto' in results.columns:
        trading_days = CRYPTO_TRADING_DAYS
    else:
        trading_days = ETFS_TRADING_DAYS 
        
    # Initialize arrays to store weights, returns, volatilities, and Sharpe ratios
    all_weights = np.zeros((num_of_portfolios, num_selected_tickers))
    ret_arr = np.zeros(num_of_portfolios)
    vol_arr = np.zeros(num_of_portfolios)
    sharpe_ratio_arr = np.zeros(num_of_portfolios)

    # Start the simulations
    for ind in range(num_of_portfolios):
        # Calculate random weights and normalize them
        weights = np.array(np.random.random(num_selected_tickers))
        weights /= np.sum(weights)

        # Store the weights in the corresponding array
        all_weights[ind, :] = weights

        # Calculate expected returns and store them in the corresponding array
        ret_arr[ind] = np.sum((results.mean() * weights) * trading_days)

        # Calculate the volatility and store it in the corresponding array
        vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(results.cov() * trading_days, weights)))

        # Calculate Sharpe Ratio and store it in the corresponding array
        sharpe_ratio_arr[ind] = ret_arr[ind] / vol_arr[ind]

    return all_weights, ret_arr, vol_arr, sharpe_ratio_arr

In [96]:
# Call the Monte Carlo simulation function for each asset class
crypto_all_weights, crypto_ret_arr, crypto_vol_arr, crypto_sharpe_ratio_arr = monte_carlo_simulation(num_of_portfolios, crypto_num_selected_tickers, crypto_results)
stocks_all_weights, stocks_ret_arr, stocks_vol_arr, stocks_sharpe_ratio_arr = monte_carlo_simulation(num_of_portfolios, stocks_num_selected_tickers, stocks_results)
commodities_all_weights, commodities_ret_arr, commodities_vol_arr, commodities_sharpe_ratio_arr = monte_carlo_simulation(num_of_portfolios, commodities_num_selected_tickers, commodities_results)
bonds_all_weights, bonds_ret_arr, bonds_vol_arr, bonds_sharpe_ratio_arr = monte_carlo_simulation(num_of_portfolios, bonds_num_selected_tickers, bonds_results)

In [97]:
# Create data frame with the weights, the returns, the volatility, and the Sharpe Ratio for each asset class
crypto_simulations_data = [crypto_ret_arr, crypto_vol_arr, crypto_sharpe_ratio_arr, crypto_all_weights]
stocks_simulations_data = [stocks_ret_arr, stocks_vol_arr, stocks_sharpe_ratio_arr, stocks_all_weights]
commodities_simulations_data = [commodities_ret_arr, commodities_vol_arr, commodities_sharpe_ratio_arr, commodities_all_weights]
bonds_simulations_data = [bonds_ret_arr, bonds_vol_arr, bonds_sharpe_ratio_arr, bonds_all_weights]

In [98]:
# Create a DataFrame from sim data and Transpose, so it will look like our original one.
crypto_simulations_df = pd.DataFrame(data=crypto_simulations_data).T
stocks_simulations_df = pd.DataFrame(data=stocks_simulations_data).T
commodities_simulations_df = pd.DataFrame(data=commodities_simulations_data).T
bonds_simulations_df = pd.DataFrame(data=bonds_simulations_data).T

### Step : Create function that will rocess the simulations data for each asset class
The function creates a summary of portfolio simulation results for an asset class, including expected returns, volatilities, Sharpe ratios, and optimal portfolio weights. It takes the simulation results, gives the columns names, infers the data types, and identifies the portfolio with the highest Sharpe ratio and the portfolio with the minimum volatility. It returns the summary in a Pandas DataFrame format.

In [99]:
def process_simulations_df(simulations_df):
    """
    This function processes the simulations data for a single asset class.
    
    Parameters:
    - simulations_df (DataFrame): DataFrame containing the simulations data for the asset class.
    
    Returns:
    - Tuple of two Series: one for the portfolio with the maximum Sharpe Ratio, and another for the portfolio with the minimum volatility.
    """
    # Give the columns names
    simulations_df.columns = [
        'Returns',
        'Volatility',
        'Sharpe Ratio',
        'Portfolio Weights'
    ]

    # Make sure the data types are correct
    simulations_df = simulations_df.infer_objects()
    
    # Find the Max Sharpe Ratio to find a better portfolio that provides the largest risk-adjusted returns
    max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

    # Find the minimum volatility from the simulations to identify a portfolio that takes on the least amount of risk
    min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]

    # Create Series for the rows with the maximum Sharpe Ratio and the minimum volatility
    max_sharpe_ratio_row = pd.Series({
        'Returns': max_sharpe_ratio['Returns'],
        'Volatility': max_sharpe_ratio['Volatility'],
        'Sharpe Ratio': max_sharpe_ratio['Sharpe Ratio'],
        'Portfolio Weights': max_sharpe_ratio['Portfolio Weights']
    })

    min_volatility_row = pd.Series({
        'Returns': min_volatility['Returns'],
        'Volatility': min_volatility['Volatility'],
        'Sharpe Ratio': min_volatility['Sharpe Ratio'],
        'Portfolio Weights': min_volatility['Portfolio Weights']
    })
    
    return max_sharpe_ratio_row, min_volatility_row, simulations_df

In [100]:
#Call the process_simulations_df() function for all 4 asset classes:
crypto_max_sharpe_ratio, crypto_min_volatility, crypto_processed_df = process_simulations_df(crypto_simulations_df)
stocks_max_sharpe_ratio, stocks_min_volatility, stocks_processed_df = process_simulations_df(stocks_simulations_df)
commodities_max_sharpe_ratio, commodities_min_volatility, commodities_processed_df = process_simulations_df(commodities_simulations_df)
bonds_max_sharpe_ratio, bonds_min_volatility, bonds_processed_df = process_simulations_df(bonds_simulations_df)

In [101]:
display(crypto_processed_df.head())
crypto_processed_df.tail()

,Returns,Volatility,Sharpe Ratio,Portfolio Weights
0,0.529326,0.896921,0.590159,"[0.14890262410546995, 0.17387985608178957, 0.1564677979660313, 0.13013379814303783, 0.16786263171818963, 0.14912722329615688, 0.07362606868932492]"
1,0.640941,0.951043,0.673934,"[0.18641917201856636, 0.21861886073720538, 0.14957039946039763, 0.014136096563750353, 0.12826742777986458, 0.11401797546121849, 0.1889700679789974]"
2,0.607334,0.829114,0.732510,"[0.0705570083401662, 0.24384817320270535, 0.19381993365712152, 0.09241211296978831, 0.1255688541495358, 0.20552076750349044, 0.0682731501771924]"
3,0.916162,1.065832,0.859574,"[0.18158863277815748, 0.21128320318196095, 0.40333908286708053, 0.03743875441755562, 0.07964776094536712, 0.05596673465682212, 0.03073583115305611]"
4,0.470866,0.941083,0.500345,"[0.22992868782247688, 0.037542253486741464, 0.13269403050407935, 0.1500862773145335, 0.17972928597535723, 0.14168126915733448, 0.12833819573947705]"


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
2995,0.483056,0.864947,0.558480,"[0.2375261715169807, 0.186842241695892, 0.07927070111679044, 0.0298514872168103, 0.10920082424491463, 0.19276458095398144, 0.1645439932546306]"
2996,0.544917,0.863418,0.631116,"[0.08250781623444961, 0.1827460803825122, 0.17544472277907006, 0.28085403982681983, 0.03612237961495924, 0.1628185897004434, 0.07950637146174566]"
2997,0.275018,0.803463,0.342291,"[0.21139380586401155, 0.05891608235852339, 0.05839722892747236, 0.1514643152912625, 0.22160915767914477, 0.2647211214440117, 0.03349828843557368]"
2998,0.619623,0.950510,0.651884,"[0.086305439316247, 0.1249592866998748, 0.1535401603420929, 0.14916682837772832, 0.14935739120564115, 0.11279406952014337, 0.2238768245382724]"
2999,0.600689,0.972610,0.617605,"[0.11778009294190799, 0.12135137169408947, 0.13935201640165437, 0.23660715649020853, 0.06743240648064364, 0.07901050124639504, 0.23846645474510098]"


In [102]:
crypto_min_volatility

Returns                                                                                                                                                            0.197916
Volatility                                                                                                                                                         0.375642
Sharpe Ratio                                                                                                                                                       0.526873
Portfolio Weights    [0.0002031731568338602, 0.011989393833844715, 0.06395473322451703, 0.18276883618410664, 0.016950724103627857, 0.6683110332973251, 0.05582210619974478]
dtype: object

In [103]:
crypto_max_sharpe_ratio

Returns                                                                                                                                                        1.026162
Volatility                                                                                                                                                     1.012855
Sharpe Ratio                                                                                                                                                   1.013138
Portfolio Weights    [0.009740904127508828, 0.4147730113517108, 0.3945729889961357, 0.010100961297739743, 0.0336532320223408, 0.06578525015507791, 0.07137365204948629]
dtype: object

In [104]:
display(stocks_processed_df.head())
stocks_processed_df.tail()

,Returns,Volatility,Sharpe Ratio,Portfolio Weights
0,0.151191,0.195281,0.774222,"[0.04811129262929683, 0.17711421477031852, 0.021560817897685425, 0.2589926878935889, 0.2172259400840009, 0.13465991043298514, 0.10742116020215697, 0.034913976089967255]"
1,0.081500,0.179249,0.454678,"[0.17408289697125254, 0.1377572917240466, 0.004587840036901701, 0.04367339638341979, 0.18091558714404166, 0.15517168203438542, 0.06493871686620692, 0.23887258883974544]"
2,0.094093,0.170333,0.552410,"[0.09158658566215232, 0.12317091622939802, 0.11686530344022365, 0.14647773799520725, 0.032354163677064215, 0.16343043917968886, 0.13258893001054284, 0.1935259238057229]"
3,0.113164,0.175865,0.643469,"[0.12268819218529577, 0.011008475967746538, 0.10605594308570342, 0.19571970617565515, 0.07800633751541111, 0.1721246859861357, 0.14727442910531857, 0.16712222997873388]"
4,0.093101,0.185373,0.502236,"[0.21701112344392318, 0.03236704237365288, 0.11697365813690377, 0.1254621538491113, 0.16789384885057407, 0.16710059671063524, 0.08307539580037358, 0.09011618083482596]"


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
2995,0.049138,0.171837,0.285958,"[0.28842288996731436, 0.05335086682099781, 0.1456286272103929, 0.05749251550595346, 0.05371582335904061, 0.0944562365092349, 0.3030671451965198, 0.0038658954305461385]"
2996,0.120339,0.179776,0.669382,"[0.0833928467612849, 0.023050941572483077, 0.11175581397845694, 0.21335004881693362, 0.16099289403711017, 0.1480627347194318, 0.1728974731079644, 0.08649724700633503]"
2997,0.126544,0.173861,0.727847,"[0.09785424304505908, 0.06329431216335552, 0.01372450868394193, 0.21308901253868687, 0.04649474956197149, 0.15364469879954085, 0.1978858802732532, 0.21401259493419106]"
2998,0.061810,0.174077,0.355075,"[0.08707109139044386, 0.1792059954545466, 0.13466246214001043, 0.035865492371336564, 0.18816883882344562, 0.1484481268866067, 0.10391200871076926, 0.12266598422284085]"
2999,0.063287,0.167527,0.377776,"[0.05052972924882182, 0.1318114783409832, 0.14241982959646704, 0.07057865084202447, 0.10317277199787248, 0.18255405386967816, 0.23602768160634988, 0.08290580449780298]"


In [105]:
stocks_min_volatility

Returns                                                                                                                                                                            0.026742
Volatility                                                                                                                                                                         0.145592
Sharpe Ratio                                                                                                                                                                       0.183681
Portfolio Weights    [0.02252045251998229, 0.11016088129010455, 0.2407086119127552, 0.0041366699657404, 0.03305203366578105, 0.02789523726083911, 0.24895866222349702, 0.31256745116130025]
dtype: object

In [106]:
stocks_max_sharpe_ratio

Returns                                                                                                                                                                                0.18035
Volatility                                                                                                                                                                            0.204169
Sharpe Ratio                                                                                                                                                                          0.883339
Portfolio Weights    [0.11509213462304181, 0.01986739057587726, 0.014943954937792797, 0.34157556285909946, 0.11520285123569059, 0.20132241652130323, 0.03388243945700964, 0.15811324979018518]
dtype: object

In [107]:
display(commodities_processed_df.head())
commodities_processed_df.tail()

,Returns,Volatility,Sharpe Ratio,Portfolio Weights
0,0.192959,0.216934,0.889482,"[0.14908130901601288, 0.23007147361505112, 0.05393122440045987, 0.24175868457302477, 0.2091220538657533, 0.1160352545296981]"
1,0.080427,0.268048,0.300048,"[0.10612179981058677, 0.271074608691414, 0.06504462406219172, 0.2425189444742485, 0.01253909951653061, 0.3027009234450283]"
2,0.201699,0.257797,0.782395,"[0.15575040921411432, 0.021123437207142843, 0.05938262880569875, 0.18731937893925696, 0.38312065944344836, 0.1933034863903387]"
3,0.113092,0.237314,0.476550,"[0.0974651853537992, 0.1751260424831235, 0.2452931692572109, 0.03258611902302113, 0.21484864150871957, 0.23468084237412568]"
4,0.188480,0.215461,0.874775,"[0.09515911329191395, 0.21730477789125122, 0.15424371942003143, 0.1989810228172174, 0.22990926017476437, 0.10440210640482167]"


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
2995,0.148550,0.221227,0.671481,"[0.1878792728272197, 0.15776795701631327, 0.10711902000699959, 0.16447829695589075, 0.1955280506549752, 0.1872274025386015]"
2996,0.108440,0.170435,0.636254,"[0.4003194207755814, 0.12981879641291774, 0.34310103417291443, 0.02869375893165563, 0.05292117908356673, 0.045145810623363765]"
2997,0.237746,0.208281,1.141467,"[0.15576899533302943, 0.31344741559967354, 0.08490532305563867, 0.12073341912939504, 0.3070485536348099, 0.01809629324745348]"
2998,0.160764,0.188347,0.853552,"[0.2408337754914201, 0.22240905996296095, 0.2209342641398855, 0.023160293882757255, 0.21184480577685383, 0.08081780074612241]"
2999,0.154748,0.262461,0.589605,"[0.0364498528383261, 0.19790471200506343, 0.09466747302558996, 0.105781863241838, 0.3030134282144586, 0.2621826706747239]"


In [108]:
commodities_max_sharpe_ratio

Returns                                                                                                                                     0.259335
Volatility                                                                                                                                  0.215451
Sharpe Ratio                                                                                                                                1.203682
Portfolio Weights    [0.2928620686676094, 0.15775990180080662, 0.05168317496785409, 0.09961146064586147, 0.39681819582283356, 0.0012651980950348445]
dtype: object

In [109]:
commodities_min_volatility

Returns                                                                                                                                    0.155571
Volatility                                                                                                                                 0.157709
Sharpe Ratio                                                                                                                               0.986443
Portfolio Weights    [0.5614823190374284, 0.1439771298937165, 0.12383515623365351, 0.035426728203499916, 0.11979609218481485, 0.015482574446886768]
dtype: object

In [110]:
display(bonds_processed_df.head())
bonds_processed_df.tail()

,Returns,Volatility,Sharpe Ratio,Portfolio Weights
0,-0.066053,0.068908,-0.958559,"[0.1500930662516976, 0.18703312079742784, 0.2229086181790612, 0.2534413393562069, 0.01252478583082961, 0.17399906958477682]"
1,-0.098966,0.084987,-1.164484,"[0.14357885326207867, 0.011819173552411038, 0.19330701504909414, 0.1916403253895628, 0.28744778497789214, 0.17220684776896128]"
2,-0.077046,0.072960,-1.056011,"[0.20382040134282842, 0.14505486381641722, 0.2149810189703464, 0.08813314909042488, 0.10652901075264468, 0.24148155602733837]"
3,-0.080933,0.074135,-1.091685,"[0.1659958921460094, 0.23185070843570627, 0.16574952895735537, 0.19024877294755174, 0.2174736204280714, 0.028681477085305954]"
4,-0.098473,0.087076,-1.130879,"[0.03686281370964161, 0.06669012361593912, 0.2710235445575236, 0.2602486867518472, 0.23768283234619672, 0.12749199901885175]"


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
2995,-0.077701,0.070125,-1.108026,"[0.2087733985242159, 0.11716234094637734, 0.17236094844893157, 0.1726109101679603, 0.14066515012599637, 0.18842725178651848]"
2996,-0.090789,0.077688,-1.168637,"[0.17115158425513263, 0.005024129820986403, 0.11347361254096648, 0.24302234145496027, 0.20847672588151236, 0.2588516060464418]"
2997,-0.063186,0.059230,-1.066792,"[0.28496367614920237, 0.03832725687029917, 0.17477254529972597, 0.2584378166424707, 0.04698291796225492, 0.19651578707604683]"
2998,-0.073468,0.068296,-1.075727,"[0.26342099702707644, 0.14922960965762344, 0.21485205930123621, 0.08467017035891586, 0.1463415065621055, 0.1414856570930425]"
2999,-0.091420,0.081036,-1.128146,"[0.17581119339158482, 0.13969464048571148, 0.20470229262589512, 0.05370440544687959, 0.23833621423857923, 0.18775125381134972]"


In [111]:
bonds_max_sharpe_ratio

Returns                                                                                                                                  -0.06317
Volatility                                                                                                                               0.082298
Sharpe Ratio                                                                                                                            -0.767582
Portfolio Weights    [0.11897455522478993, 0.452451578127112, 0.318646301997976, 0.019950242444074975, 0.009315644867023791, 0.08066167733902341]
dtype: object

In [112]:
bonds_min_volatility

Returns                                                                                                                                 -0.034869
Volatility                                                                                                                               0.036481
Sharpe Ratio                                                                                                                            -0.955805
Portfolio Weights    [0.5166870131270213, 0.09877002045127506, 0.01834678747202589, 0.3078958487179891, 0.007080226069309649, 0.0512201041623791]
dtype: object

In [113]:
def create_portfolio_weights_df(selected_tickers, max_sharpe_ratio, min_volatility):
    """
    This function creates a DataFrame with the portfolio weights for the selected tickers for both the
    maximum Sharpe ratio and minimum volatility portfolios for a single asset class.
    
    Parameters:
    - selected_tickers (list): List of selected tickers for the asset class.
    - max_sharpe_ratio (Series): Series with the maximum Sharpe ratio portfolio weights for the asset class.
    - min_volatility (Series): Series with the minimum volatility portfolio weights for the asset class.
    
    Returns:
    - Tuple of two DataFrames: one for the maximum Sharpe ratio portfolio weights and another for the minimum volatility
    portfolio weights.
    """
    
    # Create a dictionary with the portfolio weights for each portfolio
    portfolio_weights_dict = {
        'Max Sharpe Ratio': max_sharpe_ratio,
        'Min Volatility': min_volatility
    }
    
    # Create empty dictionaries for the portfolio weights
    max_sharpe_ratio_weights = {}
    min_volatility_weights = {}
    
    # Iterate over each portfolio
    for portfolio_name, portfolio_weights in portfolio_weights_dict.items():
        
        # Create an empty dictionary to store the portfolio weights for each ticker
        ticker_weights = {}
        
        # Iterate over each ticker
        for i, ticker in enumerate(selected_tickers):
            
            # Get the weight for the ticker from the portfolio weights
            ticker_weight = portfolio_weights[i]
            
            # Add the ticker weight to the ticker_weights dictionary
            ticker_weights[ticker] = ticker_weight
        
        # Convert the ticker_weights dictionary to a Series and add it to the corresponding weights dictionary
        if portfolio_name == 'Max Sharpe Ratio':
            max_sharpe_ratio_weights = pd.Series(ticker_weights, name=portfolio_name)
        else:
            min_volatility_weights = pd.Series(ticker_weights, name=portfolio_name)
    
    # Create DataFrames from the weights dictionaries
    max_sharpe_ratio_weights_df = pd.DataFrame(max_sharpe_ratio_weights)
    min_volatility_weights_df = pd.DataFrame(min_volatility_weights)
    
    # # Add a column to the DataFrames for the total weight of the portfolio
    # max_sharpe_ratio_weights_df['Total Weight'] = max_sharpe_ratio_weights_df.sum(axis=1)
    # min_volatility_weights_df['Total Weight'] = min_volatility_weights_df.sum(axis=1)
    
    # # Add a column to the DataFrames for the weight of each ticker as a percentage of the total portfolio weight
    # max_sharpe_ratio_weights_df = max_sharpe_ratio_weights_df.apply(lambda x: x/max_sharpe_ratio_weights_df['Total Weight'])
    # min_volatility_weights_df = min_volatility_weights_df.apply(lambda x: x/min_volatility_weights_df['Total Weight'])
    
    return max_sharpe_ratio_weights_df, min_volatility_weights_df

In [114]:

# Create DataFrame with portfolio weights for each asset class
crypto_max_sharpe_ratio_weights, crypto_min_volatility_weights = create_portfolio_weights_df(crypto_selected_tickers, crypto_max_sharpe_ratio['Portfolio Weights'], crypto_min_volatility['Portfolio Weights'])
stocks_max_sharpe_ratio_weights, stocks_min_volatility_weights = create_portfolio_weights_df(stocks_selected_tickers, stocks_max_sharpe_ratio['Portfolio Weights'], stocks_min_volatility['Portfolio Weights'])
commodities_max_sharpe_ratio_weights, commodities_min_volatility_weights = create_portfolio_weights_df(commodities_selected_tickers, commodities_max_sharpe_ratio['Portfolio Weights'], commodities_min_volatility['Portfolio Weights'])
bonds_max_sharpe_ratio_weights, bonds_min_volatility_weights = create_portfolio_weights_df(bonds_selected_tickers, bonds_max_sharpe_ratio['Portfolio Weights'], bonds_min_volatility['Portfolio Weights'])

In [115]:
crypto_max_sharpe_ratio_weights

,Max Sharpe Ratio
AAVE-USD,0.009741
ETH-USD,0.414773
MATIC-USD,0.394573
MKR-USD,0.010101
NEAR-USD,0.033653
PAXG-USD,0.065785
SOL-USD,0.071374


In [116]:
crypto_min_volatility_weights

,Min Volatility
AAVE-USD,0.000203
ETH-USD,0.011989
MATIC-USD,0.063955
MKR-USD,0.182769
NEAR-USD,0.016951
PAXG-USD,0.668311
SOL-USD,0.055822


In [117]:
stocks_max_sharpe_ratio_weights

,Max Sharpe Ratio
VB,0.115092
VOO,0.019867
VWO,0.014944
XLE,0.341576
XLF,0.115203
XLK,0.201322
XLU,0.033882
XLV,0.158113


In [118]:
stocks_min_volatility_weights

,Min Volatility
VB,0.022520
VOO,0.110161
VWO,0.240709
XLE,0.004137
XLF,0.033052
XLK,0.027895
XLU,0.248959
XLV,0.312567


In [119]:
commodities_max_sharpe_ratio_weights

,Max Sharpe Ratio
DBA,0.292862
JJG,0.157760
PPLT,0.051683
REMX,0.099611
UGA,0.396818
UNG,0.001265


In [120]:
commodities_min_volatility_weights

,Min Volatility
DBA,0.561482
JJG,0.143977
PPLT,0.123835
REMX,0.035427
UGA,0.119796
UNG,0.015483


In [121]:
bonds_max_sharpe_ratio_weights

,Max Sharpe Ratio
FLOT,0.118975
HYG,0.452452
PFF,0.318646
TIP,0.019950
TLT,0.009316
VWOB,0.080662


In [122]:
bonds_min_volatility_weights

,Min Volatility
FLOT,0.516687
HYG,0.098770
PFF,0.018347
TIP,0.307896
TLT,0.007080
VWOB,0.051220


In [123]:
#Assigns the maximum Sharpe ratio weights for all assets
crypto_max_weights = crypto_max_sharpe_ratio.loc['Portfolio Weights']
stocks_max_weights = stocks_max_sharpe_ratio.loc['Portfolio Weights']
commodities_max_weights = commodities_max_sharpe_ratio.loc['Portfolio Weights']
bonds_max_weights = bonds_max_sharpe_ratio.loc['Portfolio Weights']


In [124]:
#Assigns the Min volatility weights for all assets
crypto_min_vol_weights = crypto_min_volatility.loc['Portfolio Weights']
stocks_min_vol_weights = stocks_min_volatility.loc['Portfolio Weights']
commodities_min_vol_weights = commodities_min_volatility.loc['Portfolio Weights']
bonds_min_vol_weights = bonds_min_volatility.loc['Portfolio Weights']


In [125]:
print(crypto_max_weights)
print(stocks_max_weights)
print(commodities_max_weights)
print(bonds_max_weights)

[0.0097409  0.41477301 0.39457299 0.01010096 0.03365323 0.06578525
 0.07137365]
[0.11509213 0.01986739 0.01494395 0.34157556 0.11520285 0.20132242
 0.03388244 0.15811325]
[0.29286207 0.1577599  0.05168317 0.09961146 0.3968182  0.0012652 ]
[0.11897456 0.45245158 0.3186463  0.01995024 0.00931564 0.08066168]


In [126]:
print(crypto_min_vol_weights)
print(stocks_min_vol_weights)
print(commodities_min_vol_weights)
print(bonds_min_vol_weights)

[2.03173157e-04 1.19893938e-02 6.39547332e-02 1.82768836e-01
 1.69507241e-02 6.68311033e-01 5.58221062e-02]
[0.02252045 0.11016088 0.24070861 0.00413667 0.03305203 0.02789524
 0.24895866 0.31256745]
[0.56148232 0.14397713 0.12383516 0.03542673 0.11979609 0.01548257]
[0.51668701 0.09877002 0.01834679 0.30789585 0.00708023 0.0512201 ]


In [127]:
# Assign weights to asset classes for max sharpe ratio portfolio 
crypto_max_comb_df = crypto_results @ crypto_max_weights
stocks_max_comb_df = stocks_results @ stocks_max_weights
commodities_max_comb_df = commodities_results @ commodities_max_weights
bonds_max_comb_df = bonds_results @ bonds_max_weights


In [128]:
# Assign weights to asset classes for min volatility portfolio
crypto_min_comb_df = crypto_results @ crypto_min_vol_weights
stocks_min_comb_df = stocks_results @ stocks_min_vol_weights
commodities_min_comb_df = commodities_results @ commodities_min_vol_weights
bonds_min_comb_df = bonds_results @ bonds_min_vol_weights

In [129]:
display(crypto_max_comb_df.head())
display(stocks_max_comb_df.head())
display(commodities_max_comb_df.head())
display(bonds_max_comb_df.head())

Date
2020-10-15   -0.007982
2020-10-16   -0.050585
2020-10-17    0.016400
2020-10-18    0.017007
2020-10-19   -0.023580
dtype: float64

Date
2020-10-15    0.003536
2020-10-16   -0.006671
2020-10-19   -0.017340
2020-10-20    0.006735
2020-10-21   -0.009298
dtype: float64

Date
2020-10-15   -0.001399
2020-10-16   -0.009026
2020-10-19   -0.000866
2020-10-20    0.012599
2020-10-21   -0.010651
dtype: float64

Date
2020-10-15   -0.000637
2020-10-16   -0.001592
2020-10-19   -0.003534
2020-10-20    0.003580
2020-10-21   -0.002923
dtype: float64

In [130]:
display(crypto_min_comb_df.head())
display(stocks_min_comb_df.head())
display(commodities_min_comb_df.head())
display(bonds_min_comb_df.head())

Date
2020-10-15   -0.002618
2020-10-16   -0.017627
2020-10-17    0.004302
2020-10-18    0.008976
2020-10-19   -0.007368
dtype: float64

Date
2020-10-15   -0.004172
2020-10-16    0.006410
2020-10-19   -0.010913
2020-10-20    0.005249
2020-10-21   -0.002421
dtype: float64

Date
2020-10-15    0.002689
2020-10-16   -0.005391
2020-10-19    0.001111
2020-10-20    0.006932
2020-10-21   -0.000082
dtype: float64

Date
2020-10-15   -0.000591
2020-10-16    0.000061
2020-10-19   -0.001516
2020-10-20    0.000270
2020-10-21   -0.000391
dtype: float64

In [131]:
# Concatenate the four asset classes dataframes for the maximum Sharpe ratio portfolio
max_comb_df = pd.concat([crypto_max_comb_df, stocks_max_comb_df, commodities_max_comb_df, bonds_max_comb_df], axis=1)

# Add prefixes to the column names to signify the corresponding asset class
max_comb_df.columns = ['Crypto', 'Stocks', 'Commodities', 'Bonds']

# Concatenate the four asset class dataframes for the minimum volatility portfolio
min_comb_df = pd.concat([crypto_min_comb_df, stocks_min_comb_df, commodities_min_comb_df, bonds_min_comb_df], axis=1)

# Add prefixes to the column names to signify the corresponding asset class
min_comb_df.columns = ['Crypto', 'Stocks', 'Commodities', 'Bonds']

In [132]:
max_comb_df.head()

,Crypto,Stocks,Commodities,Bonds
Date,,,,
2020-10-15,-0.007982,0.003536,-0.001399,-0.000637
2020-10-16,-0.050585,-0.006671,-0.009026,-0.001592
2020-10-17,0.016400,NaN,NaN,NaN
2020-10-18,0.017007,NaN,NaN,NaN
2020-10-19,-0.023580,-0.017340,-0.000866,-0.003534


In [133]:
min_comb_df.head()

,Crypto,Stocks,Commodities,Bonds
Date,,,,
2020-10-15,-0.002618,-0.004172,0.002689,-0.000591
2020-10-16,-0.017627,0.006410,-0.005391,0.000061
2020-10-17,0.004302,NaN,NaN,NaN
2020-10-18,0.008976,NaN,NaN,NaN
2020-10-19,-0.007368,-0.010913,0.001111,-0.001516


In [134]:
# Fill NaN values in a DataFrame using forward-fill method.
max_comb_df = max_comb_df.fillna(method='ffill')
min_comb_df = min_comb_df.fillna(method='ffill')

In [135]:
max_comb_df.head() 

,Crypto,Stocks,Commodities,Bonds
Date,,,,
2020-10-15,-0.007982,0.003536,-0.001399,-0.000637
2020-10-16,-0.050585,-0.006671,-0.009026,-0.001592
2020-10-17,0.016400,-0.006671,-0.009026,-0.001592
2020-10-18,0.017007,-0.006671,-0.009026,-0.001592
2020-10-19,-0.023580,-0.017340,-0.000866,-0.003534


In [136]:
min_comb_df.head()

,Crypto,Stocks,Commodities,Bonds
Date,,,,
2020-10-15,-0.002618,-0.004172,0.002689,-0.000591
2020-10-16,-0.017627,0.006410,-0.005391,0.000061
2020-10-17,0.004302,0.006410,-0.005391,0.000061
2020-10-18,0.008976,0.006410,-0.005391,0.000061
2020-10-19,-0.007368,-0.010913,0.001111,-0.001516


In [137]:
# Define the number of selected asset classes as number of columns 
max_comb_num_selected_tickers = max_comb_df.shape[1]
min_comb_num_selected_tickers = min_comb_df.shape[1]

# Define the df that will be passed to MC simulation
max_comb_results = max_comb_df
min_comb_results = min_comb_df

# Perform Monte Carlo simulation for maximum Sharpe ratio portfolio
max_comb_all_weights, max_comb_ret_arr, max_comb_vol_arr, max_comb_sharpe_ratio_arr = monte_carlo_simulation(num_of_portfolios, max_comb_num_selected_tickers, max_comb_results)

# Perform Monte Carlo simulation for minimum volatility portfolio
min_comb_all_weights, min_comb_ret_arr, min_comb_vol_arr, min_comb_sharpe_ratio_arr = monte_carlo_simulation(num_of_portfolios, min_comb_num_selected_tickers, min_comb_results)

In [138]:
# Create data frame with the weights, the returns, the volatility, and the Sharpe Ratio arrays
# for minimum volatility and maximun sharpe ratio portfoilos for asset classes
max_comb_sim_data = [max_comb_ret_arr, max_comb_vol_arr, max_comb_sharpe_ratio_arr, max_comb_all_weights]
min_comb_sim_data = [min_comb_ret_arr, min_comb_vol_arr, min_comb_sharpe_ratio_arr, min_comb_all_weights]

In [139]:
# Create a DataFrame from sim data and Transpose, so it will look like our original one.
max_comb_simulations_df = pd.DataFrame(data=max_comb_sim_data).T
min_comb_simulations_df = pd.DataFrame(data=min_comb_sim_data).T


In [140]:
#Call the process_simulations_df() function for minimum volatility and maximun sharpe ratio portfoilos for asset classes
max_comb_max_sharpe_ratio, max_comb_min_volatility, max_comb_processed_df = process_simulations_df(max_comb_simulations_df)
min_comb_max_sharpe_ratio, min_comb_min_volatility, min_comb_processed_df = process_simulations_df(min_comb_simulations_df)


In [141]:
display(max_comb_processed_df.head())
max_comb_processed_df.tail()


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
0,0.343986,0.335617,1.024937,"[0.29552526886693764, 0.15251747588645678, 0.09696436591907914, 0.45499288932752646]"
1,0.678344,0.569583,1.190948,"[0.5321032792658756, 0.2284143364733008, 0.17168417481890674, 0.06779820944191671]"
2,0.532554,0.451116,1.180525,"[0.41630307878837913, 0.09000687018765459, 0.23048917486652346, 0.26320087615744286]"
3,0.299457,0.211994,1.412573,"[0.14526232524277632, 0.035275493497813996, 0.3912404864436355, 0.42822169481577416]"
4,0.520574,0.355860,1.462860,"[0.29261429308589904, 0.2487887631344042, 0.3562158917010088, 0.10238105207868795]"


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
2995,0.292216,0.192634,1.516947,"[0.05722351048516755, 0.37290536704178734, 0.3419162878482876, 0.22795483462475755]"
2996,0.194787,0.220968,0.881520,"[0.17602261572298428, 0.07408978363379615, 0.11941710611872736, 0.6304704945244921]"
2997,0.215677,0.185693,1.161470,"[0.07113664824423158, 0.42621761153991544, 0.14460437216430896, 0.35804136805154396]"
2998,0.416797,0.321957,1.294574,"[0.270497135752056, 0.1693755499776726, 0.26574385610374457, 0.2943834581665269]"
2999,0.202898,0.170243,1.191815,"[0.03884299307610546, 0.3902598950430676, 0.2089031836407612, 0.36199392824006577]"


In [142]:
# Review portfolio of maximun sharpe ratio for asset classes with highest Sharpe ratio
max_comb_max_sharpe_ratio

Returns                                                                                         0.503537
Volatility                                                                                      0.259145
Sharpe Ratio                                                                                    1.943073
Portfolio Weights    [0.12325464432398137, 0.10073784180312374, 0.7568184471877458, 0.01918906668514908]
dtype: object

In [143]:
# Review portfolio of minimum volatility for asset classes with highest Sharpe ratio (probably not needed)
max_comb_min_volatility

Returns                                                                                           -0.01256
Volatility                                                                                        0.106522
Sharpe Ratio                                                                                     -0.117914
Portfolio Weights    [0.032515950066617345, 0.012711156489099708, 0.07567807428725444, 0.8790948191570285]
dtype: object

In [144]:
display(min_comb_processed_df.head())
min_comb_processed_df.tail()

,Returns,Volatility,Sharpe Ratio,Portfolio Weights
0,0.092153,0.147932,0.622942,"[0.27954296074490914, 0.33399243329413514, 0.10310153626605494, 0.28336306969490077]"
1,0.105418,0.118624,0.888679,"[0.1967250622217442, 0.03660472351365089, 0.3624829833901785, 0.4041872308744264]"
2,0.076305,0.119077,0.640805,"[0.22810031201718972, 0.16747774673683294, 0.15680494645192505, 0.4476169947940522]"
3,0.171719,0.160903,1.067222,"[0.23589697795877174, 0.15738591752865738, 0.5485981162908994, 0.05811898822167147]"
4,0.150838,0.173150,0.871144,"[0.3125560766510936, 0.33507725945849076, 0.3142611958076752, 0.038105468082740436]"


,Returns,Volatility,Sharpe Ratio,Portfolio Weights
2995,0.110769,0.143190,0.773580,"[0.28893841353214356, 0.13557521849162268, 0.2553415583391917, 0.32014480963704195]"
2996,0.053601,0.121624,0.440711,"[0.24773572578086087, 0.19936734019472022, 0.030725844802426943, 0.522171089221992]"
2997,0.169829,0.244632,0.694223,"[0.5935949709293271, 0.14220203351102126, 0.2084162541746744, 0.05578674138497719]"
2998,0.179083,0.160525,1.115608,"[0.1884843036068093, 0.2069165901842891, 0.6029921706474032, 0.0016069355614984995]"
2999,0.091398,0.108706,0.840786,"[0.11231731963394984, 0.267346584601948, 0.28661230260062365, 0.33372379316347855]"


In [145]:
# Review portfolio of minimum volatility for asset classes with highest Sharpe ratio 
min_comb_max_sharpe_ratio

Returns                                                                                         0.195446
Volatility                                                                                      0.166493
Sharpe Ratio                                                                                    1.173898
Portfolio Weights    [0.12442407268508737, 0.05032479662226221, 0.797755747125353, 0.027495383567297433]
dtype: object

In [146]:
# Review portfolio of minimum volatility for asset classes with lowest volatility (probably not needed)
min_comb_min_volatility

Returns                                                                                         -0.008964
Volatility                                                                                       0.054511
Sharpe Ratio                                                                                    -0.164439
Portfolio Weights    [0.06614465857886764, 0.03291342569044224, 0.014322213665195184, 0.8866197020654949]
dtype: object

In [147]:
# # Define the number of weights for asset classes alocation
# max_comb_selected_tickers = max_comb_df.shape[1]
# min_comb_selected_tickers = min_comb_df.shape[1]

# Define list of asset classes for allocation 
max_comb_selected_tickers = max_comb_df.columns.tolist()
min_comb_selected_tickers = min_comb_df.columns.tolist()



In [148]:
max_comb_selected_tickers

['Crypto', 'Stocks', 'Commodities', 'Bonds']

In [149]:
min_comb_selected_tickers

['Crypto', 'Stocks', 'Commodities', 'Bonds']

In [150]:
# Create DataFrame with portfolio weights for each asset class
max_comb_sharpe_ratio_weights, max_comb_min_volatility_weights = create_portfolio_weights_df(max_comb_selected_tickers, max_comb_max_sharpe_ratio['Portfolio Weights'], max_comb_min_volatility['Portfolio Weights'])
min_comb_sharpe_ratio_weights, min_comb_min_volatility_weights = create_portfolio_weights_df(min_comb_selected_tickers, min_comb_max_sharpe_ratio['Portfolio Weights'], min_comb_min_volatility['Portfolio Weights'])


In [151]:
# Review df with weights for asset classes of max Sharpe ratio portfolio with highest Sharpe ratio
max_comb_sharpe_ratio_weights.T

,Crypto,Stocks,Commodities,Bonds
Max Sharpe Ratio,0.123255,0.100738,0.756818,0.019189


In [152]:
# Review df with weights for asset classes of max Sharpe ratio portfolio with lowest volatility (probably not needed)
max_comb_min_volatility_weights.T


,Crypto,Stocks,Commodities,Bonds
Min Volatility,0.032516,0.012711,0.075678,0.879095


In [153]:
# Review df with weights for asset classes of minimum volatility with highest Sharpe ratio
min_comb_sharpe_ratio_weights.T

,Crypto,Stocks,Commodities,Bonds
Max Sharpe Ratio,0.124424,0.050325,0.797756,0.027495


In [154]:
# Review df with weights for asset classes of max minimum volatility portfolio with lowest volatility (probably not needed)
min_comb_min_volatility_weights.T

,Crypto,Stocks,Commodities,Bonds
Min Volatility,0.066145,0.032913,0.014322,0.88662


In [155]:
# def calculate_final_ticker_weights(asset_class_weights_df, ticker_weights_by_asset_class):
#     # Initialize an empty dictionary to store the final ticker weights
#     final_ticker_weights = {}

#     # Iterate through each asset class in the asset_class_weights_df
#     for asset_class in asset_class_weights_df.columns:
#         # Get the weight of the asset class in the portfolio
#         asset_class_weight = asset_class_weights_df.loc['Max Sharpe Ratio', asset_class]
        
#         # Get the ticker weights for the asset class from the ticker_weights_by_asset_class dictionary
#         ticker_weights = ticker_weights_by_asset_class[asset_class]

#         # Iterate through each ticker and its weight in the asset class
#         for ticker, weight in ticker_weights.items():
#             # Calculate the final weight of the ticker in the portfolio
#             final_weight = weight * asset_class_weight
            
#             # Store the final weight in the final_ticker_weights dictionary
#             final_ticker_weights[ticker] = final_weight

#     # Return the final_ticker_weights dictionary
#     return final_ticker_weights

In [156]:
max_sharpe_portfolio = {
    "asset_class_weights": max_comb_sharpe_ratio_weights.T,
    "asset_classes": {
        "crypto": crypto_max_sharpe_ratio_weights,
        "stocks": stocks_max_sharpe_ratio_weights,
        "commodities": commodities_max_sharpe_ratio_weights,
        "bonds": bonds_max_sharpe_ratio_weights
    }
}

In [157]:
min_volatility_portfolio = {
    "asset_class_weights": min_comb_sharpe_ratio_weights.T,
    "asset_classes" : {
            'crypto': crypto_min_volatility_weights,
            'stocks': stocks_min_volatility_weights,
            'commodities': commodities_min_volatility_weights,
            'bonds': bonds_min_volatility_weights
    }
}

In [158]:
max_sharpe_portfolio

{'asset_class_weights':                     Crypto    Stocks  Commodities     Bonds
 Max Sharpe Ratio  0.123255  0.100738     0.756818  0.019189,
 'asset_classes': {'crypto':            Max Sharpe Ratio
  AAVE-USD           0.009741
  ETH-USD            0.414773
  MATIC-USD          0.394573
  MKR-USD            0.010101
  NEAR-USD           0.033653
  PAXG-USD           0.065785
  SOL-USD            0.071374,
  'stocks':       Max Sharpe Ratio
  VB            0.115092
  VOO           0.019867
  VWO           0.014944
  XLE           0.341576
  XLF           0.115203
  XLK           0.201322
  XLU           0.033882
  XLV           0.158113,
  'commodities':       Max Sharpe Ratio
  DBA           0.292862
  JJG           0.157760
  PPLT          0.051683
  REMX          0.099611
  UGA           0.396818
  UNG           0.001265,
  'bonds':       Max Sharpe Ratio
  FLOT          0.118975
  HYG           0.452452
  PFF           0.318646
  TIP           0.019950
  TLT           0.009316


In [159]:
min_volatility_portfolio

{'asset_class_weights':                     Crypto    Stocks  Commodities     Bonds
 Max Sharpe Ratio  0.124424  0.050325     0.797756  0.027495,
 'asset_classes': {'crypto':            Min Volatility
  AAVE-USD         0.000203
  ETH-USD          0.011989
  MATIC-USD        0.063955
  MKR-USD          0.182769
  NEAR-USD         0.016951
  PAXG-USD         0.668311
  SOL-USD          0.055822,
  'stocks':       Min Volatility
  VB          0.022520
  VOO         0.110161
  VWO         0.240709
  XLE         0.004137
  XLF         0.033052
  XLK         0.027895
  XLU         0.248959
  XLV         0.312567,
  'commodities':       Min Volatility
  DBA         0.561482
  JJG         0.143977
  PPLT        0.123835
  REMX        0.035427
  UGA         0.119796
  UNG         0.015483,
  'bonds':       Min Volatility
  FLOT        0.516687
  HYG         0.098770
  PFF         0.018347
  TIP         0.307896
  TLT         0.007080
  VWOB        0.051220}}

In [160]:
def calculate_final_allocation(portfolio):
    final_allocation = {}
    asset_class_weights = portfolio["asset_class_weights"]
    
    for asset_class, ticker_weights in portfolio["asset_classes"].items():
        # Match the asset class key with the corresponding column name
        asset_class_column = asset_class.title()
        
        # Calculate the final allocation for each ticker by multiplying the ticker weight
        # with the respective asset class weight
        asset_class_weight = asset_class_weights.loc[:, asset_class_column].values[0]
        final_ticker_weights = ticker_weights.multiply(asset_class_weight, axis=0)
        # Add the final ticker weights to the final_allocation dictionary
        final_allocation[asset_class] = final_ticker_weights

    return final_allocation

In [161]:
max_sharpe_final_allocation = calculate_final_allocation(max_sharpe_portfolio)
min_volatility_final_allocation = calculate_final_allocation(min_volatility_portfolio)

In [162]:
max_sharpe_final_allocation

{'crypto':            Max Sharpe Ratio
 AAVE-USD           0.001201
 ETH-USD            0.051123
 MATIC-USD          0.048633
 MKR-USD            0.001245
 NEAR-USD           0.004148
 PAXG-USD           0.008108
 SOL-USD            0.008797,
 'stocks':       Max Sharpe Ratio
 VB            0.011594
 VOO           0.002001
 VWO           0.001505
 XLE           0.034410
 XLF           0.011605
 XLK           0.020281
 XLU           0.003413
 XLV           0.015928,
 'commodities':       Max Sharpe Ratio
 DBA           0.221643
 JJG           0.119396
 PPLT          0.039115
 REMX          0.075388
 UGA           0.300319
 UNG           0.000958,
 'bonds':       Max Sharpe Ratio
 FLOT          0.002283
 HYG           0.008682
 PFF           0.006115
 TIP           0.000383
 TLT           0.000179
 VWOB          0.001548}

In [163]:
min_volatility_final_allocation

{'crypto':            Min Volatility
 AAVE-USD         0.000025
 ETH-USD          0.001492
 MATIC-USD        0.007958
 MKR-USD          0.022741
 NEAR-USD         0.002109
 PAXG-USD         0.083154
 SOL-USD          0.006946,
 'stocks':       Min Volatility
 VB          0.001133
 VOO         0.005544
 VWO         0.012114
 XLE         0.000208
 XLF         0.001663
 XLK         0.001404
 XLU         0.012529
 XLV         0.015730,
 'commodities':       Min Volatility
 DBA         0.447926
 JJG         0.114859
 PPLT        0.098790
 REMX        0.028262
 UGA         0.095568
 UNG         0.012351,
 'bonds':       Min Volatility
 FLOT        0.014207
 HYG         0.002716
 PFF         0.000504
 TIP         0.008466
 TLT         0.000195
 VWOB        0.001408}

In [164]:
# Maximum Sharpe Ratio portfolio DataFrames
max_sharpe_crypto_df = max_sharpe_final_allocation['crypto']
max_sharpe_stocks_df = max_sharpe_final_allocation['stocks']
max_sharpe_commodities_df = max_sharpe_final_allocation['commodities']
max_sharpe_bonds_df = max_sharpe_final_allocation['bonds']

# Minimum Volatility portfolio DataFrames
min_volatility_crypto_df = min_volatility_final_allocation['crypto']
min_volatility_stocks_df = min_volatility_final_allocation['stocks']
min_volatility_commodities_df = min_volatility_final_allocation['commodities']
min_volatility_bonds_df = min_volatility_final_allocation['bonds']


In [165]:

# Display the  DataFrames
print("Maximum Sharpe Ratio Portfolio - Crypto Allocations:")
max_sharpe_crypto_df


Maximum Sharpe Ratio Portfolio - Crypto Allocations:


,Max Sharpe Ratio
AAVE-USD,0.001201
ETH-USD,0.051123
MATIC-USD,0.048633
MKR-USD,0.001245
NEAR-USD,0.004148
PAXG-USD,0.008108
SOL-USD,0.008797


In [166]:
print("Maximum Sharpe Ratio Portfolio - Stocks Allocations:")
max_sharpe_stocks_df

Maximum Sharpe Ratio Portfolio - Stocks Allocations:


,Max Sharpe Ratio
VB,0.011594
VOO,0.002001
VWO,0.001505
XLE,0.034410
XLF,0.011605
XLK,0.020281
XLU,0.003413
XLV,0.015928


In [167]:

print("Maximum Sharpe Ratio Portfolio - Commodities Allocations:")
max_sharpe_commodities_df


Maximum Sharpe Ratio Portfolio - Commodities Allocations:


,Max Sharpe Ratio
DBA,0.221643
JJG,0.119396
PPLT,0.039115
REMX,0.075388
UGA,0.300319
UNG,0.000958


In [168]:
 
print("Maximum Sharpe Ratio Portfolio - Bonds Allocations:")
max_sharpe_bonds_df



Maximum Sharpe Ratio Portfolio - Bonds Allocations:


,Max Sharpe Ratio
FLOT,0.002283
HYG,0.008682
PFF,0.006115
TIP,0.000383
TLT,0.000179
VWOB,0.001548


In [169]:
print("Minimum Volatility Portfolio - Crypto Allocations:")
min_volatility_crypto_df


Minimum Volatility Portfolio - Crypto Allocations:


,Min Volatility
AAVE-USD,0.000025
ETH-USD,0.001492
MATIC-USD,0.007958
MKR-USD,0.022741
NEAR-USD,0.002109
PAXG-USD,0.083154
SOL-USD,0.006946


In [170]:
print("Minimum Volatility Portfolio - Stocks Allocations:")
min_volatility_stocks_df



Minimum Volatility Portfolio - Stocks Allocations:


,Min Volatility
VB,0.001133
VOO,0.005544
VWO,0.012114
XLE,0.000208
XLF,0.001663
XLK,0.001404
XLU,0.012529
XLV,0.015730


In [171]:

print("Minimum Volatility Portfolio - Commodities Allocations:")
min_volatility_commodities_df




Minimum Volatility Portfolio - Commodities Allocations:


,Min Volatility
DBA,0.447926
JJG,0.114859
PPLT,0.098790
REMX,0.028262
UGA,0.095568
UNG,0.012351


In [172]:
print("Minimum Volatility Portfolio - Bonds Allocations:")
min_volatility_bonds_df

Minimum Volatility Portfolio - Bonds Allocations:


,Min Volatility
FLOT,0.014207
HYG,0.002716
PFF,0.000504
TIP,0.008466
TLT,0.000195
VWOB,0.001408


In [174]:
benchmark_portfolio = {
    'crypto': ['BTC-USD'],
    'stocks': ['SPY'],
    'commodities': ['DBC'],
    'bonds': ['AGG']
}

benchmark_weights = pd.DataFrame({
    'crypto': [1],
    'stocks': [1],
    'commodities': [1],
    'bonds': [1]
}, index=benchmark_portfolio.keys())

In [175]:
start_date = '2020-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')

tickers = list(set(max_sharpe_crypto_df.index.tolist() + max_sharpe_stocks_df.index.tolist() +
                   max_sharpe_commodities_df.index.tolist() + max_sharpe_bonds_df.index.tolist() +
                   min_volatility_crypto_df.index.tolist() + min_volatility_stocks_df.index.tolist() +
                   min_volatility_commodities_df.index.tolist() + min_volatility_bonds_df.index.tolist() +
                   benchmark_portfolio['crypto'] + benchmark_portfolio['stocks'] +
                   benchmark_portfolio['commodities'] + benchmark_portfolio['bonds']))

price_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
price_data = price_data.fillna(method='ffill')

[*********************100%***********************]  31 of 31 completed

1 Failed download:
- XLF : No timezone found, symbol may be delisted
